In [ ]:
#!/usr/bin/env python
#!/usr/bin/env python
from flask import Flask, request, send_file, Blueprint
import os
import sys

import io
from io import BytesIO
import requests
from flask.json import jsonify
import jwt

from random import randint
import datetime

import smbclient 

import logging


from locale import normalize
import spacy

import io
import pandas as pd

from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spaczz.matcher import FuzzyMatcher

import requests
import json

from spacy.tokenizer import Tokenizer
from datetime import datetime
from datetime import date
import pyap
import usaddress
import re
import datefinder
from dateutil import parser
import string
from iteration_utilities import unique_everseen
from requests_ntlm import HttpNtlmAuth
import msoffcrypto
from zipfile import ZipFile
import traceback
import gc
import numpy as np

from validate_jwt import validate_jwt

facesheetextraction = Blueprint("facesheetextraction", __name__)


file_path = "./api/PDQ/reference/Spacy"
nlp = spacy.load(file_path + '/en_core_web_md-3.4.0')
matcher = Matcher(nlp.vocab)                    


## PART 1 of code from  here
REGEXDate="^(0[1-9]|1[012])[-/.](0[1-9]|[12][0-9]|3[01])[-/.](19|20)\d\d$"
REGEXPhone = r"\(?([2-9][0-9]{2})\)?[-.● ]([0-9]{3})[-.● ]([0-9]{4})"
REGEXPhone2 = r"\(?([2-9][0-9]{2})\)?[-.● ]?([0-9]{3})[-.● ]?([0-9]{4})"
REGEXICDCode = r"\b[01a-z][0-9][0-9ab]\.?\,?[0-9a-z]{0,4}"
REGEXICDCode_repl = r'[^A-Za-z0-9. ]+'
REGEXName=r'[^A-Za-z ,''-]+'
REGEXMemberId=r'([u-v][0-9]{10})|([u-v][0-9]{8})|([1]\d{10})'
REGEXMemberId2=r'([1]\d{8})'
REGEXAlphaNumeric=r'[^A-Za-z0-9]+'
REGEXMRN=r'[^A-Za-z0-9]+'


def getNPI(input):


    #npi = input[-10:]
    npi=(''.join(filter(str.isdigit, input)))

    if not( npi.isnumeric() and len(npi) == 10 and npi.startswith("1")):
        return(None)

    PARAMS = {'version':'2.1', 'number':npi}
    try:
        npiresponse = requests.get('https://npiregistry.cms.hhs.gov/api',params=PARAMS)

        npiresponsejson  = npiresponse.json()
        if "Errors" not in npiresponsejson and npiresponsejson['result_count'] != 0 :
            return(npiresponsejson)
        else:
            logging.info('FACE SHEET 2827 : extractFace sheet :  NPI registry Call returned errors:' + str(npiresponsejson) )
            return(None)
    except:
        logging.error('FACE SHEET 2827 : extractFace sheet :  NPI registry Call failed:'  )
        return(npi)

        #return(None)
    
def split_name(name):
    """
        Returns the names classified into firstname, lastname and middlename.

                Parameters:
                        name (str): name

                Returns:
                         name (dict): a dictionary with names first,last and middle name
        """
    name = name.strip('-')
    name = name.replace("(m)"," ")
    name = name.replace("(f)"," ")
    name= removeBrackets(name)
    name = name.replace("name "," ")
    name = name.replace(".",",")
    
    if "," in name:
        names = name.split(",")
        if len(names) >= 2:
            last_name = names[0]
            names.remove(last_name)
            first_name = ''.join([str(nm) for nm in names])
            return {'firstName': first_name.strip(), 'lastName': last_name.strip()}
    else:
        names = name.split()
        if len(names) >= 2:
            last_name = names[-1]
            names.remove(last_name)
            first_name = ' '.join([str(nm) for nm in names])
            return {'firstName': first_name.strip(), 'lastName': last_name.strip()}
        else:
            first_name = name
            last_name = name
            return {'firstName': first_name.strip(),'lastName': last_name.strip()}
    
    return {'firstName': name.strip(),'lastName': name.strip()}


def gettaxId(tin):
    tin=tin.strip()
    
    tinwords=tin.split('/')
    if len(tinwords) > 1:
        if len(tinwords[0].strip()) ==9:
            tin=tinwords[0]
        elif len(tinwords[1].strip()) ==9:    
            tin=tinwords[1]

    tin=(''.join(filter(str.isdigit, tin)))
    if tin.isnumeric() and len(tin) == 9:
        return(tin)
    

    return(None)
    
def formatMRN(mrnRaw):  
    mrnRaw=removeBrackets(mrnRaw)
    mrnRaw=mrnRaw.replace("mrn","").replace("MRN","")
    mrnwords=mrnRaw.split()
    for mrnword in mrnwords:
        mrn=re.sub(REGEXMRN,'',mrnword)
        numbers = sum(c.isdigit() for c in mrn)
        if numbers > 2:
            return(mrn)
    
    return None


def getEMR(emrText):
    emrText=str(emrText)
    if emrText == None: 
        return ("N")
    if emrText.upper().find("DO NOT REQUEST CLINICAL")  > -1 or emrText.upper().find("DO NOT CALL FOR CLINICAL") > -1: 
        return ("Y")
    if emrText.upper().strip() == "YES"  or emrText.upper().strip() == "Y": 
        return ("Y")
    if emrText.upper().strip() == "NO"   or emrText.upper().strip() == "N":
        return ("N")        

    return("N")

def formatPhone(phone):
    if phone is None:
        return(None)
    
    phoneNo = (''.join(filter(str.isdigit, phone)))
    if len(phoneNo) < 10:
        return(None)
        
    if phoneNo is not None:
        if phoneNo.startswith('91') and len(phoneNo) > 11:
            phoneNo==phoneNo[2:12]
        elif phoneNo.startswith('1'):
            phoneNo=phoneNo[1:11]
        else:
            phoneNo=phoneNo[0:10]

    return(phoneNo)

def removeBrackets(text):
    if text.find('(') != -1 and text.find(')') != -1:
        open = text.index('(')
        close = text.index(')')
        return text[:open] + text[close+1:]
    
    return(text)

def getProviderName(provName):
    
    provName = removeBrackets(provName)
    provName = re.sub(REGEXName, '', provName)
    provName = provName.replace(".","").lower().strip()
    provName = provName.replace(","," , ").strip()
    replaceitems = ['dr', 'md', 'do', 'rn', 'np ','lpn','mbbs']
    provNameNoTitle = ''
    provNames = provName.split()
    for provNamePart in provNames:
        if provNamePart not in replaceitems:
            provNameNoTitle = provNameNoTitle +' ' + provNamePart

    provName = provNameNoTitle
    if provName.endswith(","):
        provName=provName[:-1]
    if len(provName) ==0:
        return {'firstName': None,'lastName': None}
    return(split_name(provName))

def preclean_input_text_for_find_dates(text):
    cleaned_text = re.sub(r'[a-z]:\s', ' ', text, flags=re.IGNORECASE)
    cleaned_text = cleaned_text.replace("date/time","date time")
    return cleaned_text

def preclean_hospitalnames(hospName):
    hospName = hospName.replace('.',' ').replace("'","")
    return hospName    

def dateParse(input):
    
    try:
        posBracket = input.lower().find("(")
        if posBracket > -1:
            input = input[0:(posBracket-1)]

        if len(input) >= 10:
            input = input[0:10]
        
        wordsinDate = input.split()
        if len(wordsinDate) > 0:
            input = wordsinDate[0]


        today = date.today()
        Year = today.year

        dobparse = parser.parse(input)
        if dobparse.year > Year:
            dobparse = dobparse.replace(year=dobparse.year - 100)
        dobfmt = dobparse.strftime("%m/%d/%Y")
        return(dobfmt)        
    except Exception as ex :
        print(ex)
        return None

def dateExtract(text):
    extractedDate=None
    dates = datefinder.find_dates(preclean_input_text_for_find_dates(text),source=True, strict=True,index=True)
    if len(list(dates)) == 0:
        date_val = re.findall(r'\d+/\d+/\d+', text)   #To handle 2 digit years as datefinder is missing them
        if len(date_val) > 0:
            dates = datefinder.find_dates(date_val[0],source=True, strict=True,index=True)
            for date in dates:
                extractedDate = dateParse(date[1])
                return(extractedDate)
    else:
        dates = datefinder.find_dates(preclean_input_text_for_find_dates(text),source=True, strict=True,index=True)        
        for date in dates:
            extractedDate = dateParse(date[1])
            return(extractedDate)

    return(extractedDate)            

def translateStatetoCode(State):
    if len(State) == 2: 
        return State

    us_state_to_abbrev = {
    "alabama": "AL",
    "alaska": "AK",
    "arizona": "AZ",
    "arkansas": "AR",
    "california": "CA",
    "colorado": "CO",
    "connecticut": "CT",
    "delaware": "DE",
    "florida": "FL",
    "georgia": "GA",
    "hawaii": "HI",
    "idaho": "ID",
    "illinois": "IL",
    "indiana": "IN",
    "iowa": "IA",
    "kansas": "KS",
    "kentucky": "KY",
    "louisiana": "LA",
    "Maine": "ME",
    "maryland": "MD",
    "massachusetts": "MA",
    "michigan": "MI",
    "minnesota": "MN",
    "mississippi": "MS",
    "missouri": "MO",
    "montana": "MT",
    "nebraska": "NE",
    "nevada": "NV",
    "new hampshire": "NH",
    "new jersey": "NJ",
    "new mexico": "NM",
    #"new york": "NY",     could be city
    "north carolina": "NC",
    "north dakota": "ND",
    "ohio": "OH",
    "oklahoma": "OK",
    "oregon": "OR",
    "pennsylvania": "PA",
    "rhode island": "RI",
    "south carolina": "SC",
    "south dakota": "SD",
    "tennessee": "TN",
    "texas": "TX",
    "utah": "UT",
    "vermont": "VT",
    "virginia": "VA",
    "washington": "WA",
    "west virginia": "WV",
    "wisconsin": "WI",
    "wyoming": "WY",
    "district of columbia": "DC",
    "american samoa": "AS",
    "guam": "GU",
    "northern mariana islands": "MP",
    "puerto rico": "PR",
    "united states minor outlying islands": "UM",
    "u.s. virgin islands": "VI"    }

    citywithStateNames = {
        "AL city":  "alabama city",
        "AZ city":  "arizona city",
        "AK city":  "arkansas city",
        "CA city" :	"california city",
        "CA hot springs" :	"california hot springs",
        "CO city" : "colorado city",
        "CO springs" : "colorado springs",
        "DE city" :	"delaware city",
        "DE water gap" :	"delaware water gap",   
        "ID city" : "idaho city",
        "ID falls" : "idaho falls",
        "ID springs" : "idaho springs",
        "IL city" : "illinois city",
        "IO city" : "iowa city",
        "IO park" : "iowa park",        
        "ks city" :	"kansas city",
        "md heights" : "maryland heights",
        "md line" : "maryland line",
        "md center" : "maryland center",        
        "mi city" : "michingan city",        
        "mn city" : "minneapolis city", 
        "MI city" : "mississippi city",
        "mo city" : "montana city",
        "ne city" :	"nebraska city"
}

    for fullState in us_state_to_abbrev:
        State=State.lower().replace(fullState,us_state_to_abbrev[fullState])

    for cityWithStatename in citywithStateNames:
        if State.find(cityWithStatename) > -1: 
            State = State.replace(cityWithStatename,citywithStateNames[cityWithStatename])

    return(State)
    


def PreProcessforExtractAddress(pipelineRawContent):

    pipelineRawContent = translateStatetoCode(pipelineRawContent)
    if pipelineRawContent.find("#") > -1 :
        if  (pipelineRawContent.find(" suite ") == -1 and pipelineRawContent.find(" ste ") ==-1):
            pipelineRawContent = pipelineRawContent.replace("#","ste")
        else:
            pipelineRawContent = pipelineRawContent.replace("#"," ")

    pipelineRawContent = pyap.parser.AddressParser(country='US')._normalize_string(pipelineRawContent)
    pipelineRawContent = pipelineRawContent.upper().replace("|"," ").replace("#"," ").replace("."," ").replace(","," ").replace(";"," ").replace("'","")
    return(pipelineRawContent)
    #return(pipelineRawContent.upper().replace("|"," ").replace("#"," ").replace("."," ").replace(","," ").replace(";"," ").replace("'",""))


def ExtractAddress(strAddr):
    addresses = pyap.parse(strAddr, country='US')

    if addresses:
        return(addresses)
    else:
        return(None)

def SplitAddress(strAddr):
    varAddr1=""
    varState=""
    varCity=""
    varZip=""

    addrwords = strAddr.split()
    if len(addrwords) > 2 and addrwords[0].isnumeric() and  addrwords[1].isnumeric() :
        strAddr =  ' '.join(addrwords[1:])

    try:
        splitaddr = usaddress.tag(strAddr)
    except:
        return (varAddr1,varCity,varState,varZip)

    if len(splitaddr) != 2:
        return (varAddr1,varCity,varState,varZip)

    if splitaddr[1] != 'Street Address':
        return (varAddr1,varCity,varState,varZip)

    part = splitaddr[0]

    for component in part:
        if component == 'Recipient':
            continue
        if component == 'StateName':
            varState = part['StateName'].replace(',',' ').strip().upper()
            varState=(''.join(filter(str.isalpha, varState)))
            varState=translateStatetoCode(varState).upper()
        elif component == 'PlaceName':
            varCity = part['PlaceName'].replace(',',' ').strip()
        elif component == 'ZipCode':
            chkzip = (''.join(filter(str.isdigit, part['ZipCode'])))
            if len(chkzip) >= 4:
                varZip = part['ZipCode'].replace(',',' ').strip()
                varZip = varZip[0:5]

        else:
            varAddr1 = (varAddr1+" "+ part[component]).strip()
            if varAddr1.strip().endswith(","):
                varAddr1 = varAddr1[1:(len(varAddr1)-1)]
    return (varAddr1,varCity,varState,varZip)

def callGoogleMapsaddress(addrText):
    api_key = os.getenv('GOOGLE_MAPS_API_KEY')
    base_url = os.getenv('GOOGLE_MAPS_URL')          #"https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={addrText}&key={api_key}"
    try: 
        r = requests.get(endpoint)
        if r.status_code not in range(200, 299):
            return None
        results = r.json()['results'][0]
        address_components = results['address_components']
        foundHealth = False
        for address_component in address_components:
            for type in address_component['types']:
                if type.lower() == "health":
                    foundHealth = True
                    break
        
        if 'types' in results:
            for placeType in results['types']:
                if placeType.lower() == "health":
                    foundHealth = True
                    break

        if not foundHealth:
            return (None,None,None,None,None,None,None,None)

        for address_component in address_components:
            for type in address_component['types']:
                if type == "street_number":
                    street_number  = address_component['long_name'].lower()
                if type == "route":
                    route_short_name = address_component['short_name'].lower()
                    route_long_name  = address_component['long_name'].lower()
                if type == "locality":
                    locality_short_name = address_component['short_name'].lower()
                    locality_long_name = address_component['long_name'].lower()
                if type == "administrative_area_level_1":
                    administrative_area_level_1_short_name = address_component['short_name'].lower()
                    administrative_area_level_1_long_name = address_component['long_name'].lower()
                if type == "postal_code":
                    postal_code_short_name = address_component['short_name'].lower()
    except:
        return (None,None,None,None,None,None,None,None)
        
    return(street_number,route_short_name,route_long_name,locality_short_name,locality_long_name,administrative_area_level_1_short_name,administrative_area_level_1_long_name,postal_code_short_name)


def chkTokenPresent(pipelineRawContent,wordBeforecount,wordAftercount,fieldKeys,tokenLabel,tokenValue,row,offset=0):

    if row is not None:
        tokenPos = row['value']['spans'][0]['offset']
        tokenLen = row['value']['spans'][0]['length']
    else:
        tokenPos = (pipelineRawContent[offset:].find(tokenValue))+offset
        tokenLen = len(tokenValue)
    
    labels = fieldKeys[tokenLabel]
    if tokenPos > -1 :
        leftText = pipelineRawContent[0:tokenPos]
        rightText = pipelineRawContent[tokenPos+tokenLen: ]
        leftWords = leftText.split()
        wordBeforecount=wordBeforecount*-1
        leftSearchText = ' '.join(leftWords[int(wordBeforecount):])
        for label in labels:
            if leftSearchText.find(label) > -1: 
                return(label)        

        rightWords = rightText.split()
        rightSearchText = ' '.join(rightWords[:int(wordAftercount)])
        for label in labels:
            if rightSearchText.find(label) > -1: 
                return(label) 
            
    return None

def reformatCustomerID(inputId):
    outputId = re.sub(REGEXAlphaNumeric, '', inputId)
    return(outputId)

def reformatICD(icd):
    if icd.find('1') == 0 and icd.find('.') == 3:
        icd='i' + icd[1:]
    if icd.find('0') == 0 and icd.find('.') == 3:
        icd='o' + icd[1:]
 

    if len(icd) > 3 and icd.find('.') == -1:
        icd=icd[0:3] + '.' + icd[3:]
    return(icd)

def getICDCodes(pipelineRawContent): 
    icdRegexMatchReturn = []
    for match in re.finditer(REGEXICDCode, pipelineRawContent):
        reformattedICDCode = reformatICD(match.group())
        icdRegexMatchReturn.append([match.group(),reformattedICDCode,match.start()])
        pipelineRawContent = pipelineRawContent.replace(match.group(),reformattedICDCode)        
  
    return(icdRegexMatchReturn,pipelineRawContent)

def getPipeLineOCRRawContent(pipelineContext):
    pipelineContent=''   
    if (pipelineContext.get('OCRStep') is None):
        return(pipelineContent)

    pipelineExtractionReadResults = pipelineContext['OCRStep']['Result'][0]['analyzeResult']['readResults']
    for pipelineExtractionReadResult in pipelineExtractionReadResults:
        pipelineExtractionLines = pipelineExtractionReadResult['lines']
        for pipelineExtractionLine in pipelineExtractionLines:
            pipelineContent=pipelineContent + pipelineExtractionLine['text'] + ' '
    return(pipelineContent.replace("#"," ").replace(":"," ").replace(";"," ").replace("\n"," ").lower())
    #return(pipelineContent.replace(":"," ").replace(";"," ").replace("\n"," ").lower())

def getPipeLineRawContent(pipelineContext):
    
    pipelineExtractionStepResults = pipelineContext['ExtractionStep']['Result']
    for pipelineExtractionStepResult in pipelineExtractionStepResults:
        pipelineExtractionStepResults = pipelineExtractionStepResult['Result']
        for pipelineExtractionStepResult in pipelineExtractionStepResults:

            if pipelineExtractionStepResult['analyzeResult']['modelId'] == 'prebuilt-document': 
                pipelineContent = pipelineExtractionStepResult['analyzeResult']['content'] 
                #return(pipelineContent.replace("#","").replace(":"," ").replace(";"," ").replace("\n"," ").lower())
                return(pipelineContent.replace(":"," ").replace(";"," ").replace("\n"," ").lower())

def getPipeLineRawContentFirst4Pages(pipelineContext):
    pipelineContent=''
    pipelineExtractionStepResults = pipelineContext['ExtractionStep']['Result']
    for pipelineExtractionStepResult in pipelineExtractionStepResults:
        pipelineExtractionStepResults = pipelineExtractionStepResult['Result']
        for pipelineExtractionStepResult in pipelineExtractionStepResults:
            
            if pipelineExtractionStepResult['analyzeResult']['modelId'] == 'prebuilt-document': 
                pipelineContentpages = pipelineExtractionStepResult['analyzeResult']['pages'] 
                for pipelineContentpage in pipelineContentpages:
                    if pipelineContentpage['pageNumber'] < 5:
                        pipelineContentLines = pipelineContentpage['lines']
                        for pipelineContentLine in pipelineContentLines:
                            pipelineContent=pipelineContent+pipelineContentLine['content']+ ' '
                    #return(pipelineContent.replace("#","").replace(":"," ").replace(";"," ").replace("\n"," ").lower())
    
    return(pipelineContent.replace(":"," ").replace(";"," ").replace("\n"," ").lower())

def getPipelinePreBuiltkeyValuePairs(pipelineContext,nlp,fieldKeys):

    pipelinePreBuiltkeyValuePairs=[]
    pipelineExtractionStepResults = pipelineContext['ExtractionStep']['Result']
    for pipelineExtractionStepResult in pipelineExtractionStepResults:
        pipelineExtractionStepResults = pipelineExtractionStepResult['Result']
        for pipelineExtractionStepResult in pipelineExtractionStepResults:
            if pipelineExtractionStepResult['analyzeResult']['modelId'] != 'prebuilt-document': 
                continue
            PreBuiltkeyValuePairs = pipelineExtractionStepResult['analyzeResult']['keyValuePairs']

            for pipelinePreBuiltkeyValuePair in PreBuiltkeyValuePairs:

                if 'value'  not in pipelinePreBuiltkeyValuePair or pipelinePreBuiltkeyValuePair['value']['content'] is None or pipelinePreBuiltkeyValuePair['value']['content'] == "" or len(pipelinePreBuiltkeyValuePair['value']['content']) < 3:
                    continue

                if pipelinePreBuiltkeyValuePair['value']['content'] in fieldKeys['excludeGeneralWords']:
                    continue

                if 'confidence' in pipelinePreBuiltkeyValuePair and pipelinePreBuiltkeyValuePair['confidence']  < 0.35:
                    continue
            
                if (int(pipelinePreBuiltkeyValuePair['value']['boundingRegions'][0]['pageNumber']) < 5): 
                    #pipelinePreBuiltkeyValuePair['value']['content']= pipelinePreBuiltkeyValuePair['value']['content'].replace("#","").replace(":"," ").replace(";"," ").replace("\n"," ").lower().strip()
                    #pipelinePreBuiltkeyValuePair['key']['content']= pipelinePreBuiltkeyValuePair['key']['content'].replace("#","").replace(":"," ").replace(";"," ").replace("\n"," ").lower().strip()
                    pipelinePreBuiltkeyValuePair['Name']= pipelinePreBuiltkeyValuePair['key']['content'].replace("#"," ").replace(":"," ").replace(";"," ").replace("\n"," ").lower().strip()
                    pipelinePreBuiltkeyValuePair['Value']= pipelinePreBuiltkeyValuePair['value']['content'].replace("#"," ").replace(":"," ").replace(";"," ").replace("\n"," ").lower().strip()

                    pipelinePreBuiltkeyValuePairs.append(pipelinePreBuiltkeyValuePair)


    return(pipelinePreBuiltkeyValuePairs)

def getPipelineCustomkeyValuePairs(pipelineContext,fieldKeys):
    PipelineCustomKeyValuePairs=[]

    PipelineExtractedFields = pipelineContext['ExtractedFields']
    for pipelineExtractedField in PipelineExtractedFields:
        
        if str(pipelineExtractedField['CloudService']).find('2827-model') == -1 :  ##2827-model-using-templates-v1
            continue

        if 'Page' in pipelineExtractedField and int(pipelineExtractedField['Page']) > 5:
            continue

        if 'Value'  not in pipelineExtractedField or pipelineExtractedField['Value'] is None or pipelineExtractedField['Value'] == "" or len(pipelineExtractedField['Value']) < 3:
            continue

        if 'Confidence' in pipelineExtractedField and pipelineExtractedField['Confidence']  < 0.60:
            continue

        if pipelineExtractedField['Value'] in fieldKeys['excludeGeneralWords']:
            continue

        PipelineCustomKeyValuePairs.append(pipelineExtractedField)

    return(PipelineCustomKeyValuePairs)

def get_name_addr(mapped_name):
    
    name = "N/A"
    addr = "N/A"

    try:
        first_ind = re.search(r'\d',mapped_name).start()
        name = mapped_name[:first_ind].strip().upper()
        mapped_name = mapped_name[first_ind:]
        mapped_name = mapped_name.replace(r'\n', ' ').replace(r'\r', ' ').replace(r'\r\n', ' ')
       
        addr = re.sub(r'[^\x00-\x7f]', " ", mapped_name).strip().upper()

    except:
        return name, addr
    return name, addr
    
def get_state_details(state_sheets,tax_id, state):

    if state == 'CA' or state == 'FL' or state == 'PA':
        state_codes = [i for i in state_sheets.keys() if state in i]
        for stat in state_codes:
            df = state_sheets[stat]
            df = df.loc[df['TAX ID NUMBER'] == tax_id]
            if len(df) > 0:
                name, addr = get_name_addr(df)
                return name, addr, df.iloc[0]['TAX ID NUMBER'],df.iloc[0]['EMR AVAILABLE']
        return "N/A", "N/A", "N/A", "N/A"
    try:
        df = state_sheets[state]
        df = df.loc[df['TAX ID NUMBER'] == tax_id]
        if len(df) > 0:
            name, addr = get_name_addr(df)
            a = df.iloc[0]
            a = a.to_dict()
            del df
            return name, addr, a['TAX ID NUMBER'], a['EMR AVAILABLE']
            #return name, addr, df.iloc[0]['TAX ID NUMBER'], df.iloc[0]['EMR AVAILABLE']
    except: 
        return "N/A", "N/A", "N/A", "N/A"
    return "N/A", "N/A", "N/A", "N/A"

def ipcmReFormat(decrypted,fieldKeys):
#columns - #  PROV_ID ,PROV_NM,PROV_ALT_NM,PROV_TAX_ID,EMR AVAILABLE FACILITY NAME,_merge,TAX ID NUMBER

    decryptedXLS = pd.ExcelFile(decrypted)
    df_provider = pd.read_excel(decryptedXLS, sheet_name='Provider Alias')
    df_provider.drop(columns=['PROV_ID'])
    df_provider['PROV_TAX_ID']= df_provider['PROV_TAX_ID'].astype(str)
    df_provider['PROV_TAX_ID']= df_provider['PROV_TAX_ID'].str.zfill(9)

    #df_provider[pd.to_numeric(df_provider['NATL_PROV_ID'], errors='coerce').notnull()].astype(int)
    df_provider['NATL_PROV_ID'] = df_provider['NATL_PROV_ID'].astype('Int64').astype(str)
 
    ##################### Load all sheets
    state_df = pd.DataFrame()
    for sheet in decryptedXLS.sheet_names:
        state = sheet
        if sheet in fieldKeys['excludeIPCMGRidTabs']:
            continue
        if sheet  == 'Provider Alias':
            continue
        
        df = pd.read_excel(decryptedXLS, sheet_name=sheet, header=1)
        try:
            ind = df[df[df.columns[0]]=='FACILITY NAME'].index.values
        except:
            logging.info('FACE SHEET 2827 : ipcmReFormat unknown sheet name  - %s',str(sheet) ) 
            continue

        df = df[ind[0]:]
        df.columns = df.iloc[0]
        #df = df[pd.to_numeric(df['TAX ID NUMBER'], errors='coerce').notnull()]
        df['TAX ID NUMBER']= df['TAX ID NUMBER'].astype(str)
        df = df[['TAX ID NUMBER', 'EMR AVAILABLE', 'FACILITY NAME']]
        df = df.reset_index(drop=True)
        if len(df) > 0:
            df[['FACILITY NAME', 'ADDRESS']] = df.apply(
                lambda row: get_name_addr(row['FACILITY NAME']), axis=1, result_type='expand')
        else:
            continue
        if sheet.upper() == 'PUERTO RICO' :
            state = 'PR'
        if '-' in sheet:
            state = sheet.split('-')[0]
            state = state.strip()
            
        df = df.assign(STE_CD = state)
        df = df.T.drop_duplicates().T
        state_df = pd.concat([state_df, df])

    state_df = state_df.reset_index(drop=True)
    state_df.rename(columns={'TAX ID NUMBER': 'PROV_TAX_ID'}, inplace=True)
    state_df['PROV_TAX_ID']= state_df['PROV_TAX_ID'].astype(str)    
    state_df['PROV_TAX_ID']= state_df['PROV_TAX_ID'].str.zfill(9)

    combined_df = pd.merge(df_provider, state_df, on=['PROV_TAX_ID', 'STE_CD'], how='outer', indicator=True)
    combined_df= combined_df.drop_duplicates()

    combined_df['PROV_TAX_ID']= combined_df['PROV_TAX_ID'].astype(str)
    combined_df['PROV_TAX_ID']= combined_df['PROV_TAX_ID'].str.strip()

    combined_df['PROV_ALT_NM']= combined_df['PROV_ALT_NM'].astype(str)
    combined_df['PROV_ALT_NM']= preclean_hospitalnames(combined_df['PROV_ALT_NM'].str.strip())
    

    combined_df['PROV_NM']= combined_df['PROV_NM'].astype(str)
    combined_df['PROV_NM']= preclean_hospitalnames(combined_df['PROV_NM'].str.strip())

    combined_df['EMR AVAILABLE']= combined_df['EMR AVAILABLE'].astype(str)
    combined_df['EMR AVAILABLE']= combined_df['EMR AVAILABLE'].str.strip()

    combined_df['FACILITY NAME']= combined_df['FACILITY NAME'].astype(str)
    combined_df['FACILITY NAME']= preclean_hospitalnames(combined_df['FACILITY NAME'].str.strip())
    

    combined_df = combined_df.replace('nan', np.nan)

    del state_df
    del df_provider
    return (combined_df)

def ipcmRefresh(ipcmZipFile,fieldKeys):

    try:
        auth_user = os.getenv('OSCP_NP_USER')
        auth_password = os.getenv('OSCP_NP_PASS')     
        ipcmGridLocation = os.getenv('IPCM-GRID-LOCATION')

        IPCM_GRID_PASSWORD = os.getenv('IPCM-GRID-PASSWORD')
        
        ipcmGridXls = "IPCM Assignment Grid.xlsx"    
        user='internal'+ '\\' + auth_user
        #resp = requests.get(ipcmGridLocation,auth=HttpNtlmAuth(user,auth_password),verify=False)
        resp = requests.get(ipcmGridLocation,auth=HttpNtlmAuth(user,auth_password),verify='/usr/local/share/ca-certificates/ca-bundle.crt')

    #   Copy zip file from sharepoint to NAS share location     
        logging.info('FACE SHEET 2827 : ipcmRefresh saving zip file complete')
        with smbclient.open_file(ipcmZipFile, mode="wb",username=auth_user, password=auth_password,share_access='w') as ipcmZipFD:
            ipcmZipFD.write(resp.content)

        dir_path = os.path.dirname(os.path.realpath(__file__))

        output_folder = os.path.join(dir_path, "ipcm")
        ipcmxls_full_path = os.path.join(output_folder,ipcmGridXls)
        try:
            os.makedirs(output_folder)
            logging.info('FACE SHEET 2827 : ipcmRefresh unzipping  directory %s created successfully',output_folder)
        except:
            logging.info('FACE SHEET 2827 : ipcmRefresh unzipping  directory %s unsuccesfull, continuing.',output_folder)            
            pass

    #   Save Unzipped xls file in local  default path
        with smbclient.open_file(ipcmZipFile, mode="rb",username=auth_user, password=auth_password,share_access='r') as ipcmZipFD:
            with ZipFile(ipcmZipFD,'r') as zip:
                zip.extract(ipcmGridXls,output_folder)
                logging.info('FACE SHEET 2827 : ipcmRefresh unzipping  zip file complete')

    #   Decrypt XLSX with password
        decrypted = io.BytesIO()

    #   Read zip file from local path and decrypt into memory

        with open(ipcmxls_full_path, "rb") as f:
            ipcmXlsfileEncrypted = msoffcrypto.OfficeFile(f)
            ipcmXlsfileEncrypted.load_key(password=IPCM_GRID_PASSWORD)  # Use password
            ipcmXlsfileEncrypted.decrypt(decrypted)
            df_provider = ipcmReFormat(decrypted,fieldKeys)
            del decrypted
            logging.info('FACE SHEET 2827 : ipcmRefresh decrypt of xls file complete')
        
            try:
                # Deletes all temp file
                os.remove(ipcmxls_full_path) 
                logging.info('FACE SHEET 2827 : ipcmRefresh file %s removed successfully',ipcmxls_full_path)

                os.rmdir(output_folder)
                logging.info('FACE SHEET 2827 : ipcmRefresh directory %s removed successfully',output_folder)

            except:
                logging.error('FACE SHEET 2827 : ipcmRefresh directory %s removal Unexpected error - %s',output_folder,  sys.exc_info()[0])


            return(df_provider)


    except Exception as ex:
        logging.info('FACE SHEET 2827 : ipcm grid refresh failed with : %s ', str(ex))
        exc_type, exc_obj, exc_tb = sys.exc_info()
        logging.error(traceback.print_exc())
        return(None)

def readIPCM(ipcmZip,fieldKeys):
    auth_user = os.getenv('OSCP_NP_USER')
    auth_password = os.getenv('OSCP_NP_PASS')
    ipcmZipPickle = ipcmZip+".pickled"
    try:
        # if file in local path - check when last refreshed - if more than specified days, refresh local file
        today  = date.today()
        difference=100
        try:
            fileStat = smbclient.stat(ipcmZipPickle,username=auth_user, password=auth_password)
            dt = date.fromtimestamp(fileStat.st_chgtime)
            difference = abs(today-dt).days
        except:
            pass

        if difference >= 1:
            logging.info('FACE SHEET 2827 : ipcm grid needs refresh for  : %s ', str(today))
            df_provider = ipcmRefresh(ipcmZip,fieldKeys)
            logging.info('FACE SHEET 2827 : ipcm grid refresh complete ')

            if df_provider is not None:
                
                with smbclient.open_file(ipcmZipPickle, mode="wb",username=auth_user, password=auth_password,share_access="w") as ipcmPickleFD:
                    df_provider.to_pickle(ipcmPickleFD)
                    logging.info('FACE SHEET 2827 : ipcmRefresh completing saving of pickle format')
                    del df_provider
                    gc.collect()

        with smbclient.open_file(ipcmZipPickle, mode="rb",username=auth_user, password=auth_password,share_access="r") as ipcmFD:
            df= pd.read_pickle(ipcmFD)  #converters={'TAX ID NUMBER':str,'PROV_ALT_NM':str,'EMR AVAILALBLE':str,'FACILITY NAME':str,'NPI':str})
            logging.info('FACE SHEET 2827 : ipcmRefresh reading of pickle format complete') 
            return(df)

    except Exception as ex:
        logging.ERROR('FACE SHEET 2827 : ipcm grid readIPCM failed   : %s ', str(ex))
        return(None)

def readIPCMCPF(ipcmCPF,fieldKeys):
    auth_user = os.getenv('OSCP_NP_USER')
    auth_password = os.getenv('OSCP_NP_PASS')
    try:
        try:
            fileStat = smbclient.stat(ipcmCPF,username=auth_user, password=auth_password)
        except:
            return None

        with smbclient.open_file(ipcmCPF, mode="rb",username=auth_user, password=auth_password,share_access="r") as ipcmFD:
            df= pd.read_pickle(ipcmFD)  #converters={'TAX ID NUMBER':str,'PROV_ALT_NM':str,'EMR AVAILALBLE':str,'FACILITY NAME':str,'NPI':str})
            logging.info('FACE SHEET 2827 : ipcmRefresh reading of pickle format complete') 
            return(df)

    except Exception as ex:
        logging.ERROR('FACE SHEET 2827 : ipcm grid readIPCMCPF failed   : %s ', str(ex))
        return(None)


def getAuthorizationContactName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,nlp,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    authContactName = None
    for fieldKey in fieldKeys['authorizationContactName']:
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                authContactName = extractedField['Value']
                if authContactName.find('selected') > -1:
                    continue
                if authContactName.find(briaApiJson['customerDetails']['lastName']) > -1 or authContactName.find(briaApiJson['customerDetails']['firstName']) > -1:
                    continue
                
                authContactNameChk = re.sub(REGEXName, '', authContactName)
                if len(authContactNameChk) < 5:
                    continue
                if chkTokenPresent(pipelineRawContent,10,5,fieldKeys,'excludeContactName',authContactName,extractedField) is not None:
                    continue                    

                if chkTokenPresent(pipelineOCRRawContent,10,5,fieldKeys,'excludeContactName',authContactName,extractedField) is not None:
                    continue                    

                if (authContactName not in fieldKeys['excludeContactNameValue']):
                    briaApiJson['generalInformation']['authorizationContactName'] = authContactName
                    return


    if  briaApiJson['generalInformation']['authorizationContactName'] == "":                
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'authorizationContactName':
                if pipelineCustomkeyValuePair['Name'] not in fieldKeys['excludeContactNameValue']:
                    authContactNameChk = re.sub(REGEXName, '', pipelineCustomkeyValuePair['Value'])
                    if len(authContactNameChk) > 5 and (authContactNameChk not in fieldKeys['excludeContactNameValue']): 
                        briaApiJson['generalInformation']['authorizationContactName'] = authContactNameChk
                        return

    if  briaApiJson['generalInformation']['authorizationContactName'] == "":                
        for token in pipelineRawContentdoc:
            if token.ent_type_ == "PERSON" or token.ent_type_ == "ORG" and (token.text not in fieldKeys['excludeContactName']) :
                if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'authorizationContactName',token.text,None) is not None:
                    if chkTokenPresent(pipelineRawContent,10,5,fieldKeys,'excludeContactName',token.text,None) is None:
                        authContactNameChk = re.sub(REGEXName, '', token.text)
                        if len(authContactNameChk) > 5 and (authContactNameChk not in fieldKeys['excludeContactNameValue']): 
                            briaApiJson['generalInformation']['authorizationContactName'] = authContactNameChk
                            return

def getFaxNumber(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    
    for match in re.finditer(REGEXPhone, pipelineRawContent):

        if chkTokenPresent(pipelineRawContent,20,3,fieldKeys,'excludeContactPhone',match.group(),None, match.start()) is not None: 
            continue                   

        phoneNoRaw = formatPhone(str(match.group()))            

        if phoneNoRaw is None  or phoneNoRaw in fieldKeys['excludeCignaphoneno']:
            continue 

        if chkTokenPresent(pipelineRawContent,7,3,fieldKeys,'faxNumber',match.group(),None,match.start()) is not None:
            if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'fax',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'phone',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['faxNumber'] = phoneNoRaw
                    return
            if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'fax',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'phone',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['faxNumber'] = phoneNoRaw
                    return


    for match in re.finditer(REGEXPhone2, pipelineRawContent):
        if chkTokenPresent(pipelineRawContent,20,3,fieldKeys,'excludeContactPhone',match.group(),None, match.start()) is not None: 
            continue                   

        phoneNoRaw = formatPhone(str(match.group()))            

        if phoneNoRaw is None  or phoneNoRaw in fieldKeys['excludeCignaphoneno']:
            continue 
        
        if chkTokenPresent(pipelineRawContent,7,3,fieldKeys,'faxNumber',match.group(),None,match.start()) is not None:
            if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'fax',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'phone',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['faxNumber'] = phoneNoRaw
                    return
            if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'fax',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'phone',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['faxNumber'] = phoneNoRaw
                    return

    
    if briaApiJson['generalInformation']['faxNumber'] == "":
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'contactfaxNumber':
                phoneNoRaw = formatPhone(pipelineCustomkeyValuePair['Value'])
                if phoneNoRaw is not None  and phoneNoRaw not in fieldKeys['excludeCignaphoneno']:
                    briaApiJson['generalInformation']['faxNumber'] = phoneNoRaw
                    return

def getContactPhoneNumber(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    for match in re.finditer(REGEXPhone, pipelineRawContent):
        if chkTokenPresent(pipelineRawContent,20,3,fieldKeys,'excludeContactPhone',match.group(),None, match.start()) is not None: 
            continue                   

        phoneNoRaw = formatPhone(str(match.group()))            

        if phoneNoRaw is None  or phoneNoRaw in fieldKeys['excludeCignaphoneno'] or phoneNoRaw == briaApiJson['generalInformation']['faxNumber']:
            continue 

        if chkTokenPresent(pipelineRawContent,7,3,fieldKeys,'phoneNumber',match.group(),None,match.start()) is not None:
            if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'phone',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'fax',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                    return
            if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'phone',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'fax',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                    return

            if chkTokenPresent(pipelineRawContent,5,5,fieldKeys,'fax',match.group(),None,match.start()) is None:
                briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                return                    


    for match in re.finditer(REGEXPhone2, pipelineRawContent):
        if chkTokenPresent(pipelineRawContent,20,3,fieldKeys,'excludeContactPhone',match.group(),None, match.start()) is not None: 
            continue                   

        phoneNoRaw = formatPhone(str(match.group()))            

        if phoneNoRaw is None  or phoneNoRaw in fieldKeys['excludeCignaphoneno'] or phoneNoRaw == briaApiJson['generalInformation']['faxNumber']:
            continue 

        if chkTokenPresent(pipelineRawContent,7,3,fieldKeys,'phoneNumber',match.group(),None,match.start()) is not None:
            if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'phone',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'fax',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                    return
            if chkTokenPresent(pipelineRawContent,0,2,fieldKeys,'phone',match.group(),None,match.start()) is not None:
                if chkTokenPresent(pipelineRawContent,2,0,fieldKeys,'fax',match.group(),None,match.start()) is None:
                    briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                    return

    if briaApiJson['generalInformation']['phoneNumber'] == "":                
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'contactphoneNumber':
                phoneNoRaw = formatPhone(pipelineCustomkeyValuePair['Value'])
                if phoneNoRaw is not None and phoneNoRaw not in fieldKeys['excludeCignaphoneno']:
                    briaApiJson['generalInformation']['phoneNumber'] = phoneNoRaw
                    return


def getCustomerId(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    """
        Process data dict to fetch customer Id.

                Parameters:
                        briaApiJson (dict): Output Dictionary
                        data (list): Key Value Pair Data Dictionary

    """
    if pipelineCustomJson is not None:
        if not (pipelineCustomJson.get('customerUniqueId') is None):
            custId = reformatCustomerID(pipelineCustomJson['customerUniqueId'].lower())
            if pipelineRawContent.find(custId) > -1 and len(custId) > 7:
                briaApiJson['customerDetails']['customerId'] = custId
                return

    customerId=None
    for fieldKey in fieldKeys['customerId']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
  
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                customerId = reformatCustomerID(extractedField['Value'])
                
                if (customerId.startswith("1") or customerId.startswith("u")) and len(customerId) > 8 and  len(customerId) < 12 and customerId[1:].isnumeric():
                    briaApiJson['customerDetails']['customerId'] = customerId
                    return
                elif customerId.startswith("v") and len(customerId) > 8 and  len(customerId) < 12  and customerId[1:].isnumeric():
                    briaApiJson['customerDetails']['customerId'] = "u" + customerId[1:]
                    return
                else:
                    customerId = None

    for fieldKey in fieldKeys['customerIdFull']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName == fieldKey) :
                customerIdE = reformatCustomerID(extractedField['Value'])
                customerIds= re.finditer(REGEXMemberId,customerIdE)
                for customerIdIter in customerIds:
                    customerId = str(customerIdIter.group())
                    if (str(customerId).startswith("1") or str(customerId).startswith("u")) and len(customerId) > 8 and  len(customerId) < 12 and customerId[1:].isnumeric():
                        briaApiJson['customerDetails']['customerId'] = customerId[0:]
                        return
                    elif str(customerId).startswith("v") and len(customerId) > 8 and  len(customerId) < 12 and customerId[1:].isnumeric():
                        briaApiJson['customerDetails']['customerId'] = "u" + customerId[0][1:]
                        return
                    else:
                        customerId = None

    if briaApiJson['customerDetails']['customerId'] == "":
        customerIds= re.finditer(REGEXMemberId,pipelineRawContent)
        for customerIdIter in customerIds:
            customerId = str(customerIdIter.group())
            if chkTokenPresent(pipelineRawContent,20,3,fieldKeys,'customerId',customerId,None) is not None:
                briaApiJson['customerDetails']['customerId'] = customerId
                return 

    if briaApiJson['customerDetails']['customerId'] == "":
        customerIds= re.finditer(REGEXMemberId,pipelineOCRRawContent)
        for customerIdIter in customerIds:
            customerId = str(customerIdIter.group())
            if chkTokenPresent(pipelineOCRRawContent,20,3,fieldKeys,'customerId',customerId,None) is not None:
                briaApiJson['customerDetails']['customerId'] = customerId
                return 

    if briaApiJson['customerDetails']['customerId'] == "":
        customerIds= re.finditer(REGEXMemberId2,pipelineOCRRawContent)
        for customerIdIter in customerIds:
            customerId = str(customerIdIter.group())
            if chkTokenPresent(pipelineOCRRawContent,5,2,fieldKeys,'customerId',customerId,None) is not None:
                briaApiJson['customerDetails']['customerId'] = customerId
                return 

    if briaApiJson['customerDetails']['customerId'] == "":
        if not (pipelineCustomJson.get('customerUniqueId') is None):
            providerlinkId = pipelineCustomJson['customerUniqueId'].lower()
            matcher = FuzzyMatcher(nlp.vocab)
            matcher.add("ORTH", [nlp(providerlinkId)])
            matches = matcher(pipelineRawContentdoc)
            if len(matches) > 0:
                briaApiJson['customerDetails']['customerId'] = providerlinkId
                return        

def getAccountNum(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    accountNum=None
    for fieldKey in fieldKeys['accountNumber']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']            
            if (keyName.find(fieldKey) > -1 ):
                accountNum = extractedField['Value']
                accountNum=(''.join(filter(str.isdigit, accountNum)))
                if accountNum.isnumeric() and len(accountNum) >= 7 and len(accountNum) < 9:
                    briaApiJson['customerDetails']['accountNumber'] = accountNum
                    return
                else:
                    accountNum = None

    if accountNum is None:
        accountNums= re.finditer(r'^(d{7})$',pipelineRawContent)
        for accountNumIter in accountNums:
            accountNum = str(accountNumIter.group())
            if chkTokenPresent(pipelineRawContent,10,3,fieldKeys,'accountNumber',str(accountNum),None) is not None:
                briaApiJson['customerDetails']['accountNumber'] = accountNum
                return 

    if accountNum is None:
        accountNums= re.finditer(r'^(d{7})$',pipelineOCRRawContent)
        for accountNumIter in accountNums:
            accountNum = str(accountNumIter.group())
            if chkTokenPresent(pipelineRawContent,10,3,fieldKeys,'accountNumber',str(accountNum),None) is not None:
                briaApiJson['customerDetails']['accountNumber'] = accountNum
                return    

def getDOB(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    for fieldKey in fieldKeys['dateOfBirth']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                dobfmt=dateExtract(extractedField['Value'])          
                if dobfmt is not None:
                    briaApiJson['customerDetails']['dateOfBirth'] = dobfmt
                    return                

    if briaApiJson['customerDetails']['dateOfBirth'] == "":
        pipelineRawContentCleaned = preclean_input_text_for_find_dates(pipelineRawContent)
        dates = datefinder.find_dates(pipelineRawContentCleaned,source=True,strict=True,index=True)
        for date in dates:
            if chkTokenPresent(pipelineRawContentCleaned,5,3,fieldKeys,'dateOfBirth',date[1],None,date[2][0]) is not None:
                dobfmt = dateParse(date[1])
                if dobfmt is not None:
                    briaApiJson['customerDetails']['dateOfBirth'] = dobfmt
                    return                

    if briaApiJson['customerDetails']['dateOfBirth'] == "":
        pipelineRawContentCleaned = preclean_input_text_for_find_dates(pipelineOCRRawContent)
        dates = datefinder.find_dates(pipelineRawContentCleaned,source=True,strict=True,index=True)
        for date in dates:
            if chkTokenPresent(pipelineOCRRawContent,5,3,fieldKeys,'dateOfBirth',date[1],None) is not None:
                dobfmt = dateParse(date[1])
                if dobfmt is not None:
                    briaApiJson['customerDetails']['dateOfBirth'] = dobfmt
                    return  

    if pipelineCustomJson is not None:
        if not (pipelineCustomJson.get('dateOfBirth') is None):
            dobfmt = dateParse(pipelineCustomJson['dateOfBirth'])
            if dobfmt is not None:
                if pipelineRawContent.find(dobfmt) > -1:
                    briaApiJson['customerDetails']['dateOfBirth'] = dobfmt
                    return

    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'dateOfBirth':
            dobfmt = dateParse(pipelineCustomkeyValuePair['Value'])
            if dobfmt is not None:
                briaApiJson['customerDetails']['dateOfBirth'] = dobfmt
                return                    



def getPatientName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    if pipelineCustomJson is not None:
        if not (pipelineCustomJson.get('firstName') is None):
            firstName = pipelineCustomJson['firstName'].lower().strip()
            if pipelineRawContent.find(firstName) > -1 and len(firstName) > 3:
                briaApiJson['customerDetails']['firstName'] = firstName
            else:
                firstNameparts = firstName.split()
                if firstNameparts[0] is not None and len(firstNameparts[0]) > 3 and pipelineRawContent.find(firstNameparts[0] ) > -1:
                    briaApiJson['customerDetails']['firstName'] = firstName

       
    if pipelineCustomJson is not None:    
        if not (pipelineCustomJson.get('lastName') is None):
            lastName = pipelineCustomJson['lastName'].lower().strip()
            if pipelineRawContent.find(lastName) > -1 and len(lastName) > 3:
                briaApiJson['customerDetails']['lastName'] = lastName

    if briaApiJson['customerDetails']['lastName']   != "" and briaApiJson['customerDetails']['firstName'] != "":
        return


    for fieldKey in fieldKeys['patientName']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):  
                patientName = extractedField['Value']
                split_names = split_name(patientName)
                briaApiJson['customerDetails']['firstName'] = split_names['firstName']
                briaApiJson['customerDetails']['lastName']  = split_names['lastName']
                return

    for fieldKey in fieldKeys['patientNamePartial']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName==fieldKey) :  
                patientName = extractedField['Value']
                if chkTokenPresent(pipelineRawContent,2,10,fieldKeys,'dateOfBirth',patientName,extractedField) is not None:
                    split_names = split_name(patientName)
                    briaApiJson['customerDetails']['firstName'] = split_names['firstName']
                    briaApiJson['customerDetails']['lastName']  = split_names['lastName']
                    return
                if chkTokenPresent(pipelineOCRRawContent,2,10,fieldKeys,'dateOfBirth',patientName,extractedField) is not None:
                    split_names = split_name(patientName)
                    briaApiJson['customerDetails']['firstName'] = split_names['firstName']
                    briaApiJson['customerDetails']['lastName']  = split_names['lastName']
                    return                    

    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'patientName':
                split_names = split_name(pipelineCustomkeyValuePair['Value'])
                briaApiJson['customerDetails']['firstName'] = split_names['firstName']
                briaApiJson['customerDetails']['lastName']  = split_names['lastName']
                return                                


def getRequestType(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    
    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'patientType_Outpatient':  
            if (pipelineCustomkeyValuePair['Value'] == ":selected:"):
                briaApiJson['procedureInformation']['requestType']='Outpatient'
                return
        if pipelineCustomkeyValuePair['Name'] == 'patientType_inpatient':             
            if (pipelineCustomkeyValuePair['Value'] == ":selected:"):
                briaApiJson['procedureInformation']['requestType']='Inpatient'
                return

    for fieldKey in fieldKeys['patientType']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:  
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):  
                patientType = extractedField['Value']
                for fieldKey2 in fieldKeys['inpatient']:
                    if patientType.find(fieldKey2) > -1 :
                        briaApiJson['procedureInformation']['requestType']='Inpatient'
                        return

                for fieldKey2 in fieldKeys['observationStay']:
                    if patientType.find(fieldKey2) > -1 :
                        briaApiJson['procedureInformation']['requestType']='Outpatient'
                        return


    for fieldKey in fieldKeys['inpatient']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']            
            if ((keyName.find(fieldKey) > -1 ) and extractedField['Value'] ==":selected:"):  
                briaApiJson['procedureInformation']['requestType']='Inpatient'
                return

    for fieldKey in fieldKeys['observationStay']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']            
            if ((keyName.find(fieldKey) > -1 ) and extractedField['Value'] ==":selected:"):  
                briaApiJson['procedureInformation']['requestType']='Outpatient'
                return

## remove some keywords before looking for main words
    for fieldKey in fieldKeys['inPatientKeywordsExclude']: 
        pipelineRawContent = pipelineRawContent.replace(fieldKey," ")
        pipelineOCRRawContent = pipelineRawContent.replace(fieldKey," ")

    for fieldKey in fieldKeys['outPatientKeywordsExclude']: 
        pipelineRawContent = pipelineRawContent.replace(fieldKey," ")
        pipelineOCRRawContent = pipelineRawContent.replace(fieldKey," ")

    foundIP=False
    IPMatches = [IPpmatch for IPpmatch in fieldKeys['inPatientKeywordsInclude'] if IPpmatch in pipelineRawContent]
    IPMatches.extend([IPpmatch for IPpmatch in fieldKeys['inPatientKeywordsInclude'] if IPpmatch in pipelineOCRRawContent])
    if len(IPMatches) > 0:
        foundIP=True

    foundOP=False
    OPMatches = [OPpmatch for OPpmatch in fieldKeys['outPatientKeywordsInclude'] if OPpmatch in pipelineRawContent]
    OPMatches.extend([OPpmatch for OPpmatch in fieldKeys['outPatientKeywordsInclude'] if OPpmatch in pipelineOCRRawContent])
    if len(OPMatches) > 0:
        foundOP=True
    
    if foundIP and not foundOP:
        briaApiJson['procedureInformation']['requestType']='Inpatient'
        return

    if foundOP and not foundIP:
        briaApiJson['procedureInformation']['requestType']='Outpatient'
        return

    briaApiJson['procedureInformation']['requestType']='Inpatient'

def getProcedureStartDate(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    for fieldKey in fieldKeys['procedureStartDate']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            for exclKeys in fieldKeys['excludeprocedureStartDate']:
                if keyName.find(exclKeys) > -1: 
                    break
            
                if (keyName.find(fieldKey) > -1):
                    procedureStartDatefmt=dateExtract(extractedField['Value'])
                    if procedureStartDatefmt is not None:
                        briaApiJson['procedureInformation']['procedureStartDate'] = procedureStartDatefmt
                

    if briaApiJson['procedureInformation']['procedureStartDate'] == "":
        pipelineRawContent = preclean_input_text_for_find_dates(pipelineRawContent)
        dates = datefinder.find_dates(pipelineRawContent,source=True, strict=True,index=True)
        for date in dates:
            if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'procedureStartDate',date[1],None,date[2][0]) is not None:
                procedureStartDatefmt = date[0].strftime("%m/%d/%Y")
                if procedureStartDatefmt is not None:                
                    briaApiJson['procedureInformation']['procedureStartDate'] = procedureStartDatefmt
                    return 

    if briaApiJson['procedureInformation']['procedureStartDate'] == "":
        pipelineOCRRawContent = preclean_input_text_for_find_dates(pipelineRawContent)
        dates = datefinder.find_dates(pipelineOCRRawContent,source=True, strict=True,index=True)
        for date in dates:
            if chkTokenPresent(pipelineOCRRawContent,5,3,fieldKeys,'procedureStartDate',date[1],None,date[2][0]) is not None:
                procedureStartDatefmt = date[0].strftime("%m/%d/%Y")
                if procedureStartDatefmt is not None:                
                    briaApiJson['procedureInformation']['procedureStartDate'] = procedureStartDatefmt
                    return

    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'admissionDate':
            dates = datefinder.find_dates(pipelineCustomkeyValuePair['Value'],source=True, strict=True,index=True)
            for date in dates:
                procedureStartDatefmt = date[0].strftime("%m/%d/%Y")
                if procedureStartDatefmt is not None:                
                    briaApiJson['procedureInformation']['procedureStartDate'] = procedureStartDatefmt
                    return


def getMRN(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    for fieldKey in fieldKeys['MRN']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1) :
                mrn = formatMRN(extractedField['Value'])
                if mrn is not None:                        
                    briaApiJson['procedureInformation']['medicalRecordNumber'] = mrn
                    return

  
    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'medicalRecordNumber':
            mrn = formatMRN(pipelineCustomkeyValuePair['Value'])
            if mrn is not None:        
                briaApiJson['procedureInformation']['medicalRecordNumber'] = mrn
                return

    tokpattern = [
    [{"LOWER": "mrn"}, {"IS_SPACE": True,"OP":"?"},{}],
    [{"LOWER": "mrn"}, {"IS_SPACE": True,"OP":"?"},{"LOWER": "#","OP":"?"},{}],
    [{"LOWER": "mr"}, {"IS_SPACE": True,"OP":"?"},{"LOWER": "#","OP":"?"},{}],
    [{"LOWER": "mrn#"}, {"IS_SPACE": True,"OP":"?"},{}],
    [{"LOWER": "med rec "}, {"IS_SPACE": True,"OP":"?"},{}],
    [{"LOWER": "med rec #"}, {"IS_SPACE": True,"OP":"?"},{}]
    ]
    matcher = Matcher(nlp.vocab)                
    matcher.add("MRN", tokpattern)
    matches = matcher(pipelineRawContentdoc)
    for match_id, start, end in matches:
        mrnParts = (pipelineRawContentdoc[start:end].text).split()
        mrn=mrnParts[-1]
        mrn = formatMRN(mrn)
        if mrn is not None:        
            briaApiJson['procedureInformation']['medicalRecordNumber'] = mrn
            return

def getDiagnosisCodes(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,icd10_file,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    icdCodesextracted=[]
    diagnosisCodes=[]
    diagnosisCodeRawText = []
    auth_user = os.getenv('OSCP_NP_USER')
    auth_password = os.getenv('OSCP_NP_PASS')     
    with smbclient.open_file(icd10_file, mode="rb",username=auth_user, password=auth_password,share_access="r") as icd10FD:
        df_icd10 = pd.read_excel(icd10FD,converters={'DIAG_DESC':str,'DIAG_CD':str})        
    
    icdCodeDescriptions = df_icd10['DIAG_DESC'].tolist()
    phrasematcher = PhraseMatcher(nlp.vocab)
    phrasepatterns = [nlp.make_doc(icdCodeDescription) for icdCodeDescription in icdCodeDescriptions]
    phrasematcher.add("icdCodeDescriptions", phrasepatterns)
    del phrasepatterns

    pipelineRawContent=getPipeLineRawContentFirst4Pages(pipelineContextJson)  # Get only first 4 pages
    icdRegexMatches=[]
    icdRegexMatches,pipelineRawContent = getICDCodes(pipelineRawContent)  # preprocess pipelRawContent as well for searching

############ GET ALL diagnosis codes
    for icdRegexMatch in icdRegexMatches:
        icdMatches = df_icd10.loc[(df_icd10['DIAG_CD'] == icdRegexMatch[1])]
        if len(icdMatches.index) > 0:
            icdCodesextracted.append(icdRegexMatch[1])

    for fieldKey in fieldKeys['diagnosisCodes']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            diagKey = extractedField['Name']
            if extractedField['value']['boundingRegions'][0]['pageNumber'] > 4:
                continue
            if diagKey.lower().find(fieldKey) > -1 :
                diagnosisData,_ignore = getICDCodes(extractedField['Value'])
                if len(icdCodesextracted) > 0 and len(diagnosisData) > 0:
                    for icdCodeextracted in icdCodesextracted:
                        for diagnosis in diagnosisData:
                            if icdCodeextracted == diagnosis[1]:
                                diagnosisCode={}
                                diagnosisCode['name'] = ""
                                diagnosisCode['code'] = icdCodeextracted
                                diagnosisCodes.append(diagnosisCode)

    
    for icdRegexMatch in icdRegexMatches:
        if icdRegexMatch[1] in icdCodesextracted:
            if (chkTokenPresent(pipelineRawContent,25,5,fieldKeys,'diagnosisCodes',icdRegexMatch[1],None,icdRegexMatch[2]) is not None):
                diagnosisCode={}
                diagnosisCode['name'] = ""
                diagnosisCode['code'] = icdRegexMatch[1]
                diagnosisCodes.append(diagnosisCode)

############ GET ALL diagnosis code texts
    for fieldKey in fieldKeys['diagnosisCodes']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            if extractedField['value']['boundingRegions'][0]['pageNumber'] > 4:
                continue
            diagKey = extractedField['Name']
            if diagKey.lower().find(fieldKey) > -1  :
                diagnosisCodeRawText.append(extractedField['Value'].strip()) 

    matches = phrasematcher(pipelineRawContentdoc)
    spans= [pipelineRawContentdoc[start:end] for _,start,end in matches]
    for span in spacy.util.filter_spans(spans):
        diagtext = span.text.strip()  # The matched span
        if (chkTokenPresent(pipelineRawContent,25,5,fieldKeys,'diagnosisCodes',diagtext,None,span.start) is not None):
            diagnosisCodeRawText.append(diagtext.strip()) 

    diagnosisCodeCleanedText = []
    for diagnosisCodeText in diagnosisCodeRawText:
        diagnosisDataCleaned = re.sub(REGEXICDCode_repl,'',diagnosisCodeText)
        if len(icdCodesextracted) > 0:
            for icdRegexMatch in icdRegexMatches:
                diagnosisDataCleaned = diagnosisDataCleaned.replace(icdRegexMatch[0],"")
            if len(diagnosisDataCleaned) > 0:
                diagnosisCodeCleanedText.append(diagnosisDataCleaned.strip())
        else:
            diagnosisCodeCleanedText.append(diagnosisCodeText.strip())

    for diagnosisCodeCleaned in diagnosisCodeCleanedText:
        diagnosisCodeCleanedDoc=nlp(diagnosisCodeCleaned)
        matches = phrasematcher(diagnosisCodeCleanedDoc)
        spans= [diagnosisCodeCleanedDoc[start:end] for _,start,end in matches]
        for span in spacy.util.filter_spans(spans):
            diagtext = span.text.strip()  # The matched span
            diagnosisCode={}
            diagnosisCode['name'] = diagtext
            diagnosisCode['code'] = ""
            diagnosisCodes.append(diagnosisCode)
        
        if len(spans) ==0:
            maxLen=1000
            diagnosisCodeMatch=None
            for icdCodeDescription in icdCodeDescriptions:
                if icdCodeDescription.find(diagnosisCodeCleaned)> -1:
                    matchLen = len(icdCodeDescription)
                    if matchLen < maxLen:
                        diagnosisCodeMatch = icdCodeDescription
                        maxLen=matchLen
            if diagnosisCodeMatch is not None: 
                diagnosisCode={}
                diagnosisCode['name'] = diagnosisCodeMatch
                diagnosisCode['code'] = ""
                diagnosisCodes.append(diagnosisCode)
               

    if len(diagnosisCodes) > 0: 
        diagnosisCodes=list(unique_everseen(diagnosisCodes))
        briaApiJson['procedureInformation']['diagnosisCodes'].extend(diagnosisCodes)
        return
      

    if len(diagnosisCodes) == 0 :  # If still no diagnosis, pass default one
        diagnosisCode={}
        diagnosisCode['name'] = "No Diagnosis Code extracted"
        diagnosisCode['code'] = ""
        diagnosisCodes.append(diagnosisCode)
        briaApiJson['procedureInformation']['diagnosisCodes'].extend(diagnosisCodes)
        return    


def getProcedureCodes(pipelineRawContent,pipelineRawContentdoc,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
        #getProcedureCodes(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson)    
    procedureCodes=[]
    procedureCode={}
    procedureCode['name'] = "Observation"
    procedureCode['code'] = "99218"
    procedureCode['placeOfService'] = 'Hospital Outpatient'
    procedureCode['requestedUnits'] = "23"

    tokpattern = [
        [{"ORTH": "day"}, {"TEXT":{"REGEX": "\d{1}"}}],
        [{"TEXT":{"REGEX": "\d{1}"}},{"ORTH": "day"}],
        [{"ORTH": "day"}, {"TEXT":{"REGEX": "\d{2}"}}],
        [{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "day"}]
    ]
    matcher = Matcher(nlp.vocab)                
    matcher.add("Matcher_literal_DAYS", tokpattern)  
    matches = matcher(pipelineRawContentdoc)
    for match_id, start, end in matches:
        if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'requestedUnits','day',None) is not None:
            units=(''.join(filter(str.isdigit, pipelineRawContentdoc[start:end].text)))
            if units.isnumeric() and int(units) < 5:
                procedureCode['requestedUnits'] = 24* int((pipelineRawContentdoc[start:start].text))
                return

    tokpattern = [
        [{"ORTH": "hour"}, {"TEXT":{"REGEX": "\d{1}"}}],
        [{"TEXT":{"REGEX": "\d{1}"}},{"ORTH": "hour"}],
        [{"ORTH": "hr"}, {"TEXT":{"REGEX": "\d{2}"}}],
        [{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "hr"}],
        [{"ORTH": "hrs"}, {"TEXT":{"REGEX": "\d{1}"}}],
        [{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "hrs"}],

    ]
    matcher = Matcher(nlp.vocab)                
    matcher.add("Matcher_literal_Hours", tokpattern)  
    matches = matcher(pipelineRawContentdoc)
    for match_id, start, end in matches:
        if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'observationStay','hour',None) is not None:  
            units=(''.join(filter(str.isdigit, pipelineRawContentdoc[start:end].text)))          
            if units.isnumeric() and int(units) < 100:
                procedureCode['requestedUnits'] = units

    
    briaApiJson['procedureInformation']['procedureCodes'].append(procedureCode)

def getRequestingNPI(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    
    for fieldKey in fieldKeys['NPIrequesting']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                npiResponse = getNPI(extractedField['Value'])
                if npiResponse is not None and npiResponse["results"][0]["enumeration_type"] == "NPI-1":
                    briaApiJson['providerDetails']['requestingProvider']['npi'] = npiResponse["results"][0]["number"]
                    break
                else:
                    if (extractedField['Value']).isnumeric() and (extractedField['Value']).startswith("1") and len((extractedField['Value'])) ==10 :
                        briaApiJson['providerDetails']['requestingProvider']['npi'] = (extractedField['Value'])
                        break

    for fieldKey in fieldKeys['NPIservicing']:
        fieldKey = fieldKey.lower()
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                npiResponse = getNPI(extractedField['Value'])
                if npiResponse is not None and npiResponse["results"][0]["enumeration_type"] == "NPI-2":
                    briaApiJson['providerDetails']['servicingProvider']['npi'] = npiResponse["results"][0]["number"]


#IF No npi tags are found 
    for requestingNPIMatch in re.findall("[1][0-9]{9}", pipelineRawContent):
        npiRegistryResponse = getNPI(requestingNPIMatch)
        if npiRegistryResponse is None:
            continue

        if npiRegistryResponse["results"][0]["enumeration_type"] == "NPI-2":
            npiServicingProvider = npiRegistryResponse["results"][0]["number"]
            if briaApiJson['providerDetails']['servicingProvider']['npi'] == "":
                if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'NPI',npiServicingProvider,None) is not None:
                    briaApiJson['providerDetails']['servicingProvider']['npi'] = npiServicingProvider
        
        elif npiRegistryResponse["results"][0]["enumeration_type"] == "NPI-1":
            npiRequestingProvider = npiRegistryResponse["results"][0]["number"]
            npiRequestingProviderFirstName = npiRegistryResponse["results"][0]["basic"]["first_name"].lower()
            npiRequestingProviderLastName = npiRegistryResponse["results"][0]["basic"]["last_name"].lower()
            if pipelineRawContent.find(npiRequestingProviderFirstName) > -1 or pipelineRawContent.find(npiRequestingProviderLastName) > -1:
                if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'NPIrequesting',npiRequestingProvider,None) is not None:
                    briaApiJson['providerDetails']['requestingProvider']['npi'] = npiRequestingProvider
                    briaApiJson['providerDetails']['requestingProvider']['firstName'] = npiRegistryResponse["results"][0]["basic"]["first_name"]
                    briaApiJson['providerDetails']['requestingProvider']['lastName'] = npiRegistryResponse["results"][0]["basic"]["last_name"]
                    continue
                else:
                    briaApiJson['providerDetails']['requestingProvider']['npi'] = npiRequestingProvider
                    briaApiJson['providerDetails']['requestingProvider']['firstName'] = npiRegistryResponse["results"][0]["basic"]["first_name"]
                    briaApiJson['providerDetails']['requestingProvider']['lastName'] = npiRegistryResponse["results"][0]["basic"]["last_name"]

            else:
                if chkTokenPresent(pipelineRawContent,5,3,fieldKeys,'NPI',npiRequestingProvider,None) is not None:
                    briaApiJson['providerDetails']['requestingProvider']['npi'] = npiRequestingProvider


def getServicingfacilityName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,hospitalfile,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    if briaApiJson['providerDetails']['servicingProvider']['facilityName'] == "":
        phrasematcher = PhraseMatcher(nlp.vocab)

        #hospitallist = open(hospitalfile,'r')
        #phrasepatterns = [nlp.make_doc(phraseLine.lower().replace("\n","")) for phraseLine in hospitallist]
        auth_user = os.getenv('OSCP_NP_USER')
        auth_password = os.getenv('OSCP_NP_PASS')         
        with smbclient.open_file(hospitalfile, mode="r",username=auth_user, password=auth_password,share_access="r") as hospitallist:
            phrasepatterns = [nlp.make_doc(phraseLine.lower().replace("\n","")) for phraseLine in hospitallist]


        phrasematcher.add("Hospitals", phrasepatterns)
        matches = phrasematcher(pipelineRawContentdoc)
        match_count_dictionary = {} #hold counts of unique matches
        
        del phrasepatterns        
        maxlen=0
        maxStart=0
        maxEnd=0
        for match_id, start, end in matches:
            matchLen = len(str(pipelineRawContentdoc[start:end]))
            if matchLen > maxlen:
                maxlen = matchLen
                maxStart = start
                maxEnd =end
            if len(matches) > 1: #populate dictionary only if more than one match found
                if str(pipelineRawContentdoc[start:end]) in match_count_dictionary:
                    match_count_dictionary[str(pipelineRawContentdoc[start:end])] = match_count_dictionary.get(str(pipelineRawContentdoc[start:end])) + 1 #increasing count of match occurrences
                else:
                    match_count_dictionary[str(pipelineRawContentdoc[start:end])] = 1 # all unique matches start at 1
        if len(matches) > 1: # only finding the highest count match if there is more than one match found
            # edit to account for non-stem matches
            maxcount = 0 
            mostmatches = ''
            for x,y in match_count_dictionary.items():
                if y > maxcount:
                    maxcount = y
                    mostmatches = x
            if maxcount > 1: # if there is a max count that is greater than 1 
                stem = mostmatches
                maxlen = len(stem)
                stem_matches = [match for match in match_count_dictionary.keys() if match.find(stem) != -1]
                maxcount = 0
                for match in stem_matches: #checking if another match has the most common stem, but is more common
                    if len(match) > maxlen: #if the match is longer than the stem (eliminates getting the stem count)
                        if match_count_dictionary.get(match) > maxcount:
                            maxcount = match_count_dictionary.get(match)
                            mostmatches = match


        if maxlen > 0:        
            briaApiJson['providerDetails']['servicingProvider']['facilityName'] = str(pipelineRawContentdoc[maxStart:maxEnd])
            return

    if briaApiJson['providerDetails']['servicingProvider']['facilityName'] == "":
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'facilityName':
                if len(str(pipelineCustomkeyValuePair['Value'])) > len(briaApiJson['providerDetails']['servicingProvider']['facilityName']):
                    briaApiJson['providerDetails']['servicingProvider']['facilityName']  = str(pipelineCustomkeyValuePair['Value'])
                    return

    if briaApiJson['providerDetails']['servicingProvider']['facilityName'] == "":
        for fieldKey in fieldKeys['servicingProviderfacilityName']:
            fieldKey = fieldKey.lower()
            for extractedField in pipelinePreBuiltkeyValuePairs:
                keyName = extractedField['Name']
                if (keyName.find(fieldKey) > -1):
                    briaApiJson['providerDetails']['servicingProvider']['facilityName']  = str(extractedField['Value'])
                    return

def getRequestingProvider(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,nlp,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    if briaApiJson['providerDetails']['requestingProvider']['firstName'] != "" and briaApiJson['providerDetails']['requestingProvider']['lastName'] != "":
        return  

    referringProvider=[]
    for fieldKey in fieldKeys['requestingProviderName']:
        fieldKey = fieldKey.lower()

        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            keyValue = extractedField['Value']
            
            foundproviderNameExclude = False
            for providerNameExclude in fieldKeys['excludeProviderName']:
                if (providerNameExclude in keyName) or (providerNameExclude in keyValue):
                    if keyValue not in referringProvider:
                        referringProvider.append(keyValue)
                    foundproviderNameExclude = True
                    break
            if (foundproviderNameExclude):
                continue            
            
            foundDoctorTitle = False

            for drTitle in fieldKeys['doctorTitlesKey']:
                if keyName.find(drTitle) > -1: 
                    foundDoctorTitle=True
                    break 
            for drTitle in fieldKeys['doctorTitlesValue']:
                if keyValue.find(drTitle) > -1: 
                    foundDoctorTitle=True
                    break 
            
            if not(foundDoctorTitle):
                continue

            if (keyName.find(fieldKey) > -1 ):
                if str(extractedField['Value']).find("selected") > -1: 
                    break
                providerNames = getProviderName(extractedField['Value'])
                
                if  (providerNames.get('firstName') is not None):
                    briaApiJson['providerDetails']['requestingProvider']['firstName'] = providerNames['firstName']
                if ( (providerNames.get('lastName') is not None) ):                    
                    briaApiJson['providerDetails']['requestingProvider']['lastName']  = providerNames['lastName']
                    # special logic added when name is split in multiple lines fir XSOLIS templates
                    if providerNames['firstName'] == providerNames['lastName']:
                        tokenPos = extractedField['value']['spans'][0]['offset']
                        remainingText = pipelineRawContent[tokenPos:].replace(',',' ')
                        splitWords = remainingText.split()
                        if len(splitWords) >0 :
                            briaApiJson['providerDetails']['requestingProvider']['firstName'] = splitWords[1]
                    return 



    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'admitting_physician_name':
            if len(briaApiJson['providerDetails']['requestingProvider']['firstName']) + len(briaApiJson['providerDetails']['requestingProvider']['lastName']) < len(pipelineCustomkeyValuePair['Value']):
                if pipelineCustomkeyValuePair['Value'].lower() not in referringProvider: 
                    providerNames = getProviderName(str(pipelineCustomkeyValuePair['Value']))
                    briaApiJson['providerDetails']['requestingProvider']['firstName'] = providerNames['firstName']
                    briaApiJson['providerDetails']['requestingProvider']['lastName']  = providerNames['lastName']
                    return

    for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
        if pipelineCustomkeyValuePair['Name'] == 'attending_physician_name':
            if len(briaApiJson['providerDetails']['requestingProvider']['firstName']) + len(briaApiJson['providerDetails']['requestingProvider']['lastName']) < len(pipelineCustomkeyValuePair['Value']):
                if pipelineCustomkeyValuePair['Value'].lower() not in referringProvider: 
                    providerNames = getProviderName(str(pipelineCustomkeyValuePair['Value']))
                    briaApiJson['providerDetails']['requestingProvider']['firstName'] = providerNames['firstName']
                    briaApiJson['providerDetails']['requestingProvider']['lastName']  = providerNames['lastName']
                    return

    for fieldKey in fieldKeys['requestingProviderName']:
        fieldKey = fieldKey.lower()

        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            keyValue = extractedField['Value']
            
            if (keyName.find(fieldKey) > -1 ):
                if str(extractedField['Value']).find("selected") > -1: 
                    break
                providerNames = getProviderName(extractedField['Value'])
                providerNamesFirst = providerNames.get('firstName')
                providerNamesLast = providerNames.get('lastName')
                if providerNamesLast is not None and providerNamesFirst is not None :
                    providerNamesLastDoc = nlp(providerNamesLast)
                    
                    for token in providerNamesLastDoc:
                        if token.ent_type_ == "PERSON":
                            briaApiJson['providerDetails']['requestingProvider']['lastName']  = providerNamesLast
                            break
                    providerNamesFirsttDoc = nlp(providerNamesFirst)
                    for token in providerNamesFirsttDoc:
                        if token.ent_type_ == "PERSON":
                            briaApiJson['providerDetails']['requestingProvider']['firstName']  = providerNamesFirst
                            break



def getTIN(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    for fieldKey in fieldKeys['TIN']:
        fieldKey = fieldKey.lower()
       
        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            if (keyName.find(fieldKey) > -1 ):
                tin = gettaxId(extractedField['Value'])
                if tin is not None:
                    
                    if chkTokenPresent(pipelineRawContent,5,2,fieldKeys,'TINrequesting',extractedField['Value'],extractedField) is not None:
                        if briaApiJson['providerDetails']['requestingProvider']['tin'] == '':
                            briaApiJson['providerDetails']['requestingProvider']['tin'] = tin
                    else:
                        if briaApiJson['providerDetails']['servicingProvider']['tin'] == '':
                            briaApiJson['providerDetails']['servicingProvider']['tin'] = tin
                        


    tokpattern = [
    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "id"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "id"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "i"},{"LOWER": "d"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "i"},{"LOWER": "d"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "i"},{"LOWER": "d"},{"IS_SPACE": True,"OP":"?"},{},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tax"}, {"ORTH": "-","OP":"?"},{"LOWER": "i"},{"LOWER": "d"},{"IS_SPACE": True,"OP":"?"},{},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tin"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tin"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tid"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tid"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "taxid"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "taxid"}, {"ORTH": "-","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tax"},{"LOWER": "identification"},{"NORM": "number","OP":"?"},{"IS_SPACE": True,"OP":"?"}, {"ORTH": "-","OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tax"},{"LOWER": "identification"},{"NORM": "number","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"ORTH": "-","OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}],

    [{"LOWER": "tax"},{"NORM": "number","OP":"?"},{"IS_SPACE": True,"OP":"?"}, {"ORTH": "-","OP":"?"},{"TEXT":{"REGEX": "\d{9}"}}],
    [{"LOWER": "tax"},{"NORM": "number","OP":"?"},{"IS_SPACE": True,"OP":"?"},{"ORTH": "-","OP":"?"},{"TEXT":{"REGEX": "\d{2}"}},{"ORTH": "-","OP":"?"} ,{"TEXT":{"REGEX": "\d{7}"}}]

    ]
    matcher = Matcher(nlp.vocab)                
    matcher.add("TIN", tokpattern)
    matches = matcher(pipelineRawContentdoc)
    for match_id, start, end in matches:
        tin = gettaxId(pipelineRawContentdoc[start:end].text)
        if tin is not None:        
            if chkTokenPresent(pipelineRawContent,5,2,fieldKeys,'TINrequesting',tin,None) is not None:
                if chkTokenPresent(pipelineRawContent,5,2,fieldKeys,'TINservicing',tin,None) is None:
                    if briaApiJson['providerDetails']['requestingProvider']['tin'] == '':
                        briaApiJson['providerDetails']['requestingProvider']['tin'] = tin
                        continue
            if briaApiJson['providerDetails']['servicingProvider']['tin'] == '':
                briaApiJson['providerDetails']['servicingProvider']['tin'] = tin


    if briaApiJson['providerDetails']['servicingProvider']['tin'] == '':
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'servicing_Provider_Tax_ID_number':
                tin = gettaxId(pipelineCustomkeyValuePair['Value'])
                if tin is not None:
                    briaApiJson['providerDetails']['servicingProvider']['tin'] = tin
                    continue                

    if briaApiJson['providerDetails']['requestingProvider']['tin'] == '':
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'requestingprovider_tax_id_number':
                requestingTin = gettaxId(pipelineCustomkeyValuePair['Value'])
                if requestingTin is not None:
                    if (requestingTin != briaApiJson['providerDetails']['servicingProvider']['tin']):
                        briaApiJson['providerDetails']['requestingProvider']['tin'] = requestingTin
                        continue

def getStateCode(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    tokpattern = [ 
[{'LENGTH': 2,"ORTH":{"REGEX":"(a[klrz]|c[aot]|d[ce]|fl|ga|hi|i[adln]|k[sy]|la|m[adeinost]|n[cdehjmvy]|o[hkr]|pa|ri|s[cd]|t[nx]|ut|v[at]|w[aivy])"}},{"TEXT":{"REGEX": "\d{5}"}}]
]
    matcher = Matcher(nlp.vocab)                
    matcher.add("ADDRESS", tokpattern)
    matches = matcher(pipelineRawContentdoc)
    match_count_dictionary = {} #hold counts of unique matches
    for match_id, start, end in matches:
        words = str(pipelineRawContentdoc[start:end]).split()
        if len(words) > 1:
            state = words[0]
            if state in match_count_dictionary:
                match_count_dictionary[state] = match_count_dictionary.get(state) + 1 #increasing count of match occurrences
            else:
                match_count_dictionary[state] = 1 # all unique matches start at 1

    if len(matches) > 1: # only finding the highest count match if there is more than one match found
        maxcount = 0 
        mostmatches = ''
        for x,y in match_count_dictionary.items():
            if y > maxcount:
                maxcount = y
                mostmatches = x
                briaApiJson['providerDetails']['servicingProvider']['address']['state'] = mostmatches



def getAddress(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    if briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] =="":
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'facilityAddress':
                varAddr1,varCity,varState,varZip = SplitAddress(pipelineCustomkeyValuePair['Value'])
                if varAddr1 != '' and   varCity != '' and varState != '' and varZip != '':
                    briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
                    briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
                    briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
                    briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
                    return      


    for fieldKey in fieldKeys['servicingProviderfacilityName']:
        fieldKey = fieldKey.lower()

        for extractedField in pipelinePreBuiltkeyValuePairs:
            keyName = extractedField['Name']
            
            foundAddressKeyword = False
            for addressKeyWord in fieldKeys['addressKeywords']:
                if keyName.find(addressKeyWord) > -1: 
                    foundAddressKeyword=True
                    break 
            if not(foundAddressKeyword):
                continue

            if (keyName.find(fieldKey) > -1 ):
                if str(extractedField['Value']).find("selected") > -1: 
                    break

                varAddr1,varCity,varState,varZip = SplitAddress(str(extractedField['Value']).upper())
                if varAddr1 == '' or  varCity == '' or varState == '' or varZip == '':
                    continue

                briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
                briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
                briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
                briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
                return

    pipelineRawContent=PreProcessforExtractAddress(pipelineRawContent)
    addresses = ExtractAddress(pipelineRawContent)
    if addresses is None:
        return 
    match_start=0
    for address in addresses:
        if str(address).find(briaApiJson['providerDetails']['servicingProvider']['facilityName'].upper()) > -1:
            addressFmt=str(address).replace(briaApiJson['providerDetails']['servicingProvider']['facilityName'].upper()," ")
        else:
            addressFmt=str(address)

        varAddr1,varCity,varState,varZip = SplitAddress(addressFmt)

        if varAddr1 == '' or  varCity == '' or varState == '' or varZip == '':
            match_start=pipelineRawContent.find(varAddr1)
            continue

        if chkTokenPresent(pipelineRawContent,10,5,fieldKeys,'excludeAddress',str(address).upper(),None,match_start) is not None:
            match_start=pipelineRawContent.find(varAddr1)            
            continue
        
        if chkTokenPresent(pipelineRawContent,10,3,fieldKeys,'servicingProviderfacilityName',str(address).upper(),None,match_start) is not None:
            briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
            briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
            briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
            briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
            return

        facilitySearch = PreProcessforExtractAddress(briaApiJson['providerDetails']['servicingProvider']['facilityName'].lower())
        for match in re.finditer(facilitySearch, pipelineRawContent):
            facilityPositionEnd = match.end()
            facilityPositionStart = match.start()
            #addressPos = pipelineRawContent.find((str(address)).lower())
            #addressPosStart=address.match_start 
            #addressPosEnd=address.match_end
            addressPosStart=pipelineRawContent.find(varAddr1)   
            addressPosEnd=pipelineRawContent.find(varZip)+len(varZip)
            try:
                if  addressPosStart > facilityPositionStart:
                    textBetween = pipelineRawContent[facilityPositionEnd:addressPosStart]
                    if ( len(textBetween.split()) < 25) :
                        briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
                        briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
                        briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
                        briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
                        return                    
                else:
                    textBetween = pipelineRawContent[addressPosEnd :facilityPositionStart]
                    if ( len(textBetween.split()) < 10) :
                        briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
                        briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
                        briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
                        briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
                        return                    
                    
            except:
                pass

    if briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] =="":
        for pipelineCustomkeyValuePair in pipelineCustomkeyValuePairs:
            if pipelineCustomkeyValuePair['Name'] == 'facilityAddress':
                varAddr1,varCity,varState,varZip = SplitAddress(pipelineCustomkeyValuePair['Value'])
                if varAddr1 != '' and   varCity != '' and varState != '' and varZip != '':
                    briaApiJson['providerDetails']['servicingProvider']['address']['state'] = varState
                    briaApiJson['providerDetails']['servicingProvider']['address']['city'] = varCity
                    briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = varZip
                    briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = varAddr1
                    return                
    
    getStateCode(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)

def getGoogleMapsaddress(pipelineRawContent,briaApiJson):


    cntBlank = 0
    if briaApiJson['providerDetails']['servicingProvider']['facilityName'] == "":
        cntBlank = cntBlank+1

    if briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] == "":
        cntBlank = cntBlank+1

    if briaApiJson['providerDetails']['servicingProvider']['address']['city'] == "":
        cntBlank = cntBlank+1

    if briaApiJson['providerDetails']['servicingProvider']['address']['state'] == "":
        cntBlank = cntBlank+1

    if briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] == "":
        cntBlank = cntBlank+1

    if cntBlank > 2: # require atleast two pieces to proceed to google search
        return

    # Call google with Facility name and state code and get address, If address is empty but address line1, city, state is present in document , populate with what is returned by Google
    addressText =  " ".join([briaApiJson['providerDetails']['servicingProvider']['facilityName'] ,
                            briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'],
                            briaApiJson['providerDetails']['servicingProvider']['address']['city'] ,
                            briaApiJson['providerDetails']['servicingProvider']['address']['state'] ,
                            briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] ])
    (street_number,route_short_name,route_long_name,locality_short_name,\
        locality_long_name,administrative_area_level_1_short_name,\
            administrative_area_level_1_long_name,postal_code_short_name) = callGoogleMapsaddress(addressText)

    if street_number is None:
        return

    if pipelineRawContent.find(street_number) > -1:
        if (((pipelineRawContent.find(locality_short_name) > -1) or (pipelineRawContent.find(locality_long_name) > -1 )) and ((pipelineRawContent.find(administrative_area_level_1_short_name) > -1) or (pipelineRawContent.find(administrative_area_level_1_long_name) > -1 )))  \
        or (((pipelineRawContent.find(locality_short_name) > -1) or (pipelineRawContent.find(locality_long_name) > -1 )) and ((pipelineRawContent.find(postal_code_short_name) > -1))) \
        or ((pipelineRawContent.find(postal_code_short_name) > -1)                                                       and ((pipelineRawContent.find(administrative_area_level_1_short_name) > -1))): 
            briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] = " ".join([street_number,route_long_name])
            briaApiJson['providerDetails']['servicingProvider']['address']['city'] = locality_long_name
            briaApiJson['providerDetails']['servicingProvider']['address']['state'] = administrative_area_level_1_short_name.upper()
            briaApiJson['providerDetails']['servicingProvider']['address']['zipCode'] = postal_code_short_name



def getRequestingProviderAddress(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):

    if briaApiJson['providerDetails']['requestingProvider']['lastName'] =="":
        return

    pipelineRawContent=PreProcessforExtractAddress(pipelineRawContent)
    addresses = ExtractAddress(pipelineRawContent)
    provAddressAnchors=[]
    if briaApiJson['providerDetails']['requestingProvider']['lastName'] != '':
        provAddressAnchors.append(briaApiJson['providerDetails']['requestingProvider']['lastName'].lower())

    if briaApiJson['providerDetails']['requestingProvider']['npi'] != '':
        provAddressAnchors.append(briaApiJson['providerDetails']['requestingProvider']['npi'].lower())

    for providerSearch in provAddressAnchors:
        
        if addresses is None:
            return 
        match_start=0
        for address in addresses:

            varAddr1,varCity,varState,varZip = SplitAddress(str(address))

            if varAddr1 == '' or  varCity == '' or varState == '' or varZip == '':
                match_start=address.match_start
                continue

            providerSearch = PreProcessforExtractAddress(providerSearch)
            for match in re.finditer(providerSearch, pipelineRawContent):
                providerPositionEnd = match.end()
                providerPositionStart = match.start()
                #addressPos = pipelineRawContent.find((str(address)).lower())
                addressPosStart=pipelineRawContent.find(varAddr1)
                try:
                    if  addressPosStart > providerPositionStart:
                        textBetween = pipelineRawContent[providerPositionEnd:addressPosStart]
                        if ( len(textBetween.split()) < 5) :
                            briaApiJson['providerDetails']['requestingProvider']['address']['state'] = varState
                            briaApiJson['providerDetails']['requestingProvider']['address']['city'] = varCity
                            briaApiJson['providerDetails']['requestingProvider']['address']['zipCode'] = varZip
                            briaApiJson['providerDetails']['requestingProvider']['address']['addressLineOne'] = varAddr1
                            return                    
                    
                except:
                    pass

def findIPCMBestMatch(taxid,facilityName,npi,address,df):

    provMatches = pd.DataFrame()
    state = address['state'].upper()

    zipcode =  address['zipCode'][0:5]

    city =  address['city'].upper()
    addressLineOne =  address['addressLineOne'].upper()
    facilityName = facilityName.upper()

    if addressLineOne != '' :
        bldg_nm = addressLineOne.split(' ')[0]
    
    if state != '' and facilityName != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and STE_CD == @state and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('all 5 cases match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches  
    
    if state != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "STE_CD == @state and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid" 
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, address, npi match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches  
    
    if state != '' and facilityName != '' and addressLineOne != '' and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and STE_CD == @state and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, address, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
    
    if state != '' and facilityName != '' and npi.isnumeric() and taxid.isnumeric():
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and STE_CD == @state and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, npi, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
    
    if state != '' and facilityName != '' and addressLineOne != '' and npi.isnumeric() and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and STE_CD == @state and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, npi, address, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches

    if facilityName != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address, npi, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches

    if taxid.isnumeric() and state != '' and addressLineOne != '' and city != '' and zipcode != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, tax ID and state match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
    
    if taxid.isnumeric() and npi.isnumeric() and state != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID, npi and state match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric() and state != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and (PROV_TAX_ID == @taxid ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID, facility name and state match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
    
    if npi.isnumeric() and state != '' and addressLineOne != '' and city != '' and zipcode != '' :
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and (NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('Address, NPI and State match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches
    
    if npi.isnumeric() and state != '' and facilityName != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and (NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('facility name, NPI and State match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches

    if taxid.isnumeric() and npi.isnumeric() and addressLineOne != '' and city != '' and zipcode != '':
        qry = "(PROV_TAX_ID == @taxid ) and (NATL_PROV_ID == @npi) and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, tax ID and npi match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches

    if taxid.isnumeric() and addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and (PROV_TAX_ID == @taxid ) and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches

    if npi.isnumeric() and taxid.isnumeric() and facilityName != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and (NATL_PROV_ID == @npi ) and (PROV_TAX_ID == @taxid )"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, npi, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches

    if npi.isnumeric() and addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and (NATL_PROV_ID == @npi ) and ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, npi, facility name match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches
    
    if addressLineOne != '' and state != '' and facilityName != '' and city != '' and zipcode != '':
        facility_name = facilityName.split(' ')[0]
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and STE_CD == @state and `FACILITY NAME`.str.contains(@facility_name, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('Building number, state, and facility name match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches  
    
    if npi.isnumeric() and taxid.isnumeric() :
        qry = "(NATL_PROV_ID == @npi ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('NPI and Tax ID Match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches 
    
    if taxid.isnumeric() and npi.isnumeric():
        qry = "(NATL_PROV_ID == @npi ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID and npi match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches

    if taxid.isnumeric() and state != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID and state match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches
            if provMatches['ADDRESS'].nunique() == 1 :
                return provMatches                
    
    if npi.isnumeric() and state != '':
        qry = "(NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('NPI and State match')
            if provMatches['FACILITY NAME'].nunique() == 1 and provMatches['ADDRESS'].nunique() == 1 :
                return provMatches    
            if provMatches['PROV_TAX_ID'].nunique() == 1:
                return provMatches     

    if state != '' and facilityName != '':
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('facility name exact and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 : 
                return provMatches                

    if facilityName != '' and taxid.isnumeric():
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('FACILITY NAME contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric():
        qry = "`FACILITY NAME`.str.contains(@facilityName, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('FACILITY NAME contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if facilityName != '' and taxid.isnumeric():
        qry = "PROV_ALT_NM.str.contains(@facilityName, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric():
        qry = "PROV_NM.str.contains(@facilityName, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
  
    
    if facilityName != '' and npi.isnumeric():
        qry = "(`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName ) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('exact facility name match and npi match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if facilityName != '' and npi.isnumeric():
        qry = "`FACILITY NAME`.str.contains(@facilityName, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('FACILITY NAME contains facility name and npi')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if facilityName != '' and npi.isnumeric():
        qry = "PROV_ALT_NM.str.contains(@facilityName, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM contains facility name and npi')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and npi.isnumeric():
        qry = "PROV_NM.str.contains(@facilityName, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM contains facility name and npi')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and state != ''  and city != '' and zipcode != '':
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and STE_CD == @state and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('building number, state, and city match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if addressLineOne != '' and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and PROV_TAX_ID == @taxid and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and (`FACILITY NAME` == @facilityName or PROV_NM == @facilityName or PROV_ALT_NM == @facilityName )"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, facility name match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and npi.isnumeric() and city != '' and zipcode != '':
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('npi, address match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if state != '' and facilityName != '':
        qry = "`FACILITY NAME`.str.contains(@facilityName, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('FACILITY NAME contains facility name and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if state != '' and facilityName != '':
        qry = "PROV_ALT_NM.str.contains(@facilityName, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM contains facility name and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches
    
    if state != '' and facilityName != '':
        qry = "PROV_NM.str.contains(@facilityName, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM contains facility name and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and zipcode != '' and city != '':
        qry = "ADDRESS.str.startswith(@bldg_nm, na=False) and ADDRESS.str.contains(@city, na=False) and ADDRESS.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('building number, city and zipcode match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['ADDRESS'].nunique() ==1 :
                return provMatches            
           
    return(provMatches)

def findIPCMBestMatchCPF(taxid,facilityName,npi,address,df):

    provMatches = pd.DataFrame()
    state = address['state'].upper()

    zipcode =  address['zipCode'][0:5]

    city =  address['city'].upper()
    addressLineOne =  address['addressLineOne'].upper()
    facilityName = re.sub("[^0-9a-zA-Z]+", "", facilityName.upper())

    if addressLineOne != '' :
        bldg_nm = addressLineOne.split(' ')[0]

    df['PROV_NM_FMT']   = df['PROV_NM'].str.replace('\W', '', regex=True)  #temporary, can be done in df building itself
    df['PROV_ALT_NM_FMT']   = df['PROV_ALT_NM'].str.replace('\W', '', regex=True) #temporary, can be done in df building itself
    
    if state != '' and facilityName != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and STE_CD == @state and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('all 5 cases match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if state != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "STE_CD == @state and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid" 
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, address, npi match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if state != '' and facilityName != '' and addressLineOne != '' and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and STE_CD == @state and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, address, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if state != '' and facilityName != '' and npi.isnumeric() and taxid.isnumeric():
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and STE_CD == @state and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, tax id, npi, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if state != '' and facilityName != '' and addressLineOne != '' and npi.isnumeric() and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and STE_CD == @state and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('state, npi, address, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches

    if facilityName != '' and addressLineOne != '' and npi.isnumeric() and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address, npi, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches

    if taxid.isnumeric() and state != '' and addressLineOne != '' and city != '' and zipcode != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, tax ID and state match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if taxid.isnumeric() and npi.isnumeric() and state != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID, npi and state match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric() and state != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and (PROV_TAX_ID == @taxid ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID, facility name and state match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if npi.isnumeric() and state != '' and addressLineOne != '' and city != '' and zipcode != '' :
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and (NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('Address, NPI and State match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    
    
    if npi.isnumeric() and state != '' and facilityName != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and (NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('facility name, NPI and State match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    

    if taxid.isnumeric() and npi.isnumeric() and addressLineOne != '' and city != '' and zipcode != '':
        qry = "(PROV_TAX_ID == @taxid ) and (NATL_PROV_ID == @npi) and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, tax ID and npi match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches

    if taxid.isnumeric() and addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and (PROV_TAX_ID == @taxid ) and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    

    if npi.isnumeric() and taxid.isnumeric() and facilityName != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and (NATL_PROV_ID == @npi ) and (PROV_TAX_ID == @taxid )"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, npi, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    

    if npi.isnumeric() and addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and (NATL_PROV_ID == @npi ) and FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, npi, facility name match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    
    
    if addressLineOne != '' and state != '' and facilityName != '' and city != '' and zipcode != '':
        facility_name = facilityName.split(' ')[0]
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('Building number, state, and facility name match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches  
    
    if npi.isnumeric() and taxid.isnumeric() :
        qry = "(NATL_PROV_ID == @npi ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('NPI and Tax ID Match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if taxid.isnumeric() and npi.isnumeric():
        qry = "(NATL_PROV_ID == @npi ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID and npi match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches

    if taxid.isnumeric() and state != '':
        qry = "(PROV_TAX_ID == @taxid ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax ID and state match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches
    
    if npi.isnumeric() and state != '':
        qry = "(NATL_PROV_ID == @npi ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('NPI and State match')
            if provMatches['PROV_NM_FMT'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() == 1 :
                return provMatches    

    if state != '' and facilityName != '':
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('facility name exact and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 : 
                return provMatches                

    if facilityName != '' and taxid.isnumeric():
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('FACILITY NAME contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric():
        qry = "PROV_ALT_NM_FMT.str.contains(@facilityName, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM_FMT contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and taxid.isnumeric():
        qry = "PROV_NM_FMT.str.contains(@facilityName, na=False) and PROV_TAX_ID == @taxid"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM_FMT contains facility name and tax id match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
  
    
    if facilityName != '' and npi.isnumeric():
        qry = "(PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName ) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('exact facility name match and npi match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches

    if facilityName != '' and npi.isnumeric():
        qry = "PROV_ALT_NM_FMT.str.contains(@facilityName, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM_FMT contains facility name and npi')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
    
    if facilityName != '' and npi.isnumeric():
        qry = "PROV_NM_FMT.str.contains(@facilityName, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM_FMT contains facility name and npi')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and state != ''  and city != '' and zipcode != '':
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and STE_CD == @state and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('building number, state, and city match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
    
    if addressLineOne != '' and taxid.isnumeric() and city != '' and zipcode != '':
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and PROV_TAX_ID == @taxid and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('tax id, address match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and facilityName != '' and city != '' and zipcode != '':
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and (PROV_NM_FMT == @facilityName or PROV_ALT_NM_FMT == @facilityName )"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('address, facility name match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches

    if addressLineOne != '' and npi.isnumeric() and city != '' and zipcode != '':
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False) and NATL_PROV_ID == @npi"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('npi, address match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
  
    if state != '' and facilityName != '':
        qry = "PROV_ALT_NM_FMT.str.contains(@facilityName, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_ALT_NM_FMT contains facility name and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches
    
    if state != '' and facilityName != '':
        qry = "PROV_NM_FMT.str.contains(@facilityName, na=False) and STE_CD == @state"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('PROV_NM_FMT contains facility name and state match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches

     
    if addressLineOne != '' and zipcode != '' and city != '':
        qry = "FRST_LN_ADDR.str.startswith(@bldg_nm, na=False) and CITY_NM.str.contains(@city, na=False) and ZIP_CD.str.contains(@zipcode, na=False)"
        provMatches = df.query(qry)
        if len(provMatches.index > 0):
            print('building number, city and zipcode match')
            if provMatches['PROV_TAX_ID'].nunique() == 1 and provMatches['FRST_LN_ADDR'].nunique() ==1 :
                return provMatches            
           
    return(provMatches)

def getIPCMValues(ipcmgrid,briaApiJson,fieldKeys):
    
    df = readIPCM(ipcmgrid,fieldKeys)
    taxid = briaApiJson['providerDetails']['servicingProvider']['tin']
    facilityName = briaApiJson['providerDetails']['servicingProvider']['facilityName']
    npi = briaApiJson['providerDetails']['servicingProvider']['npi']
    address = briaApiJson['providerDetails']['servicingProvider']['address']

    provMatches = findIPCMBestMatch(taxid,facilityName,npi,address,df)
    briaApiJson['additionalProviderDetails']['emrAvailable'] = 'N' 
    
    if len(provMatches.index) > 0:
        
        prov_TaxID = provMatches['PROV_TAX_ID'].dropna().unique()
        if len(prov_TaxID) == 1:
            briaApiJson['additionalProviderDetails']['tin'] = prov_TaxID[0]

        provNM = provMatches['PROV_NM'].dropna().unique()
        facilityName =  provMatches['FACILITY NAME'].dropna().unique()
        if len(provNM) == 1 and   len(provNM[0].strip()) > 3 :
            briaApiJson['additionalProviderDetails']['facilityName'] = provNM[0]
        elif (len(facilityName) ==1) and (len(facilityName[0].strip())) > 3 :
            briaApiJson['additionalProviderDetails']['facilityName'] = facilityName[0]

        provaddress = provMatches['ADDRESS'].dropna().unique()
        if len(provaddress) == 1:
            ipcmAddresses = PreProcessforExtractAddress (provaddress[0])
            ipcmAddresses = ExtractAddress(ipcmAddresses)
            if ipcmAddresses is not None:
                for addr in ipcmAddresses:
                    try:
                        varAddr1,varCity,varState,varZip = SplitAddress(str(addr).upper().replace("|"," ").replace("#"," "))
                        briaApiJson['additionalProviderDetails']['address']['state'] = varState
                        briaApiJson['additionalProviderDetails']['address']['city'] = varCity
                        briaApiJson['additionalProviderDetails']['address']['zipCode'] = varZip
                        briaApiJson['additionalProviderDetails']['address']['addressLineOne'] = varAddr1
                    except Exception as ex:
                        logging.info('FACE SHEET 2827 : ipcm grid address  extraction failure : %s ', str(ex))
                        pass
                qry = "ADDRESS == @provaddress[0]"
                emrMatches = provMatches.query(qry)

                if len(emrMatches.index) > 0:
                    emr = str(emrMatches['EMR AVAILABLE'].dropna().unique()[0])
                    briaApiJson['additionalProviderDetails']['emrAvailable'] = getEMR(emr)


def getIPCMValuesCPF(ipcmgridCPF,briaApiJson,fieldKeys):
    
    df = readIPCMCPF(ipcmgridCPF,fieldKeys)
    taxid = briaApiJson['providerDetails']['servicingProvider']['tin']
    facilityName = briaApiJson['providerDetails']['servicingProvider']['facilityName']
    npi = briaApiJson['providerDetails']['servicingProvider']['npi']
    address = briaApiJson['providerDetails']['servicingProvider']['address']

    provMatches = findIPCMBestMatchCPF(taxid,facilityName,npi,address,df)
    if len(provMatches.index) > 0:
        
        prov_TaxID = provMatches['PROV_TAX_ID'].dropna().unique()
        provNM = provMatches['PROV_NM'].dropna().unique()
        provNMAlt =  provMatches['PROV_ALT_NM'].dropna().unique()
        provaddress = provMatches['FRST_LN_ADDR'].dropna().unique()
        varState = provMatches['STE_CD'].dropna().unique()
        varZip = provMatches['ZIP_CD'].dropna().unique()
        varCity = provMatches['CITY_NM'].dropna().unique()

        if len(prov_TaxID) == 1 and len(provaddress) == 1 :
            print("provaddress taken from CPF")
            briaApiJson['additionalProviderDetails']['tin'] = prov_TaxID[0]

            if len(provNM) == 1 :
                briaApiJson['additionalProviderDetails']['facilityName'] = provNM[0].lower()
            elif (len(provNMAlt) ==1):
                briaApiJson['additionalProviderDetails']['facilityName'] = provNMAlt[0].lower()

            briaApiJson['additionalProviderDetails']['address']['state'] = varState[0]
            briaApiJson['additionalProviderDetails']['address']['city'] = varCity[0]
            briaApiJson['additionalProviderDetails']['address']['zipCode'] = varZip[0]
            briaApiJson['additionalProviderDetails']['address']['addressLineOne'] = provaddress[0]


def facilitySpecifickeyValueOverrides(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,keyValueOverrides):
    for OverrideKeyValue in keyValueOverrides['OverrideKeyValues']:
        briaJsonGroup = OverrideKeyValue['briaJsonGroup']
        briaJsonSubGroup = OverrideKeyValue['briaJsonSubGroup']
        briaJsonField = OverrideKeyValue['briaJsonField']
        pipeLineContextKeyName = OverrideKeyValue['pipeLineContextKeyName']

        if briaApiJson['providerDetails']['servicingProvider']['facilityName'].find(OverrideKeyValue['facilityName']) > -1: 
            for extractedField in pipelinePreBuiltkeyValuePairs:
                keyName = extractedField['Name']
                if (keyName == pipeLineContextKeyName ):
                    if briaJsonSubGroup == None:
                        briaApiJson[briaJsonGroup][briaJsonField] = extractedField['Value']
                    else:
                        briaApiJson[briaJsonGroup][briaJsonSubGroup][briaJsonField] = extractedField['Value']
                    


def getStaticValues(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,idpRequestID,pipelineRawContentdoc,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs):
    if briaApiJson['procedureInformation']['requestType'] == 'Inpatient':
        briaApiJson['procedureInformation']['admissionType'] = 'Emergent'
        briaApiJson['procedureInformation']['placeOfService'] = 'Hospital - Inpatient'
        briaApiJson['procedureInformation']['stayLevel'] = 'Medical'
        del briaApiJson['procedureInformation']['procedureCodes']   
    elif briaApiJson['procedureInformation']['requestType'] == 'Outpatient':
        try:
            getProcedureCodes(pipelineRawContent,pipelineRawContentdoc,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
        except Exception as ex:
            message="Failure in GetProcedureCodes:" + str(ex)
            logging.warning('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',pipelineContextJson['Request']['Id'],message)
        del briaApiJson['procedureInformation']['placeOfService']
        del briaApiJson['procedureInformation']['admissionType']
        del briaApiJson['procedureInformation']['stayLevel']

    briaApiJson['procedureInformation']['serviceType'] = 'Medical'    
    briaApiJson['procedureInformation']['sheetType'] = 'facesheet'
    briaApiJson['transactionMetadata']['autoSubmit'] = 'false'    
    # TO DO write logic based on number of fields extracted
    briaApiJson['transactionMetadata']['status'] = 'SUCCESS'
    briaApiJson['transactionMetadata']['transactionId'] = pipelineContextJson['Request']['Id']
    #if message is None:
    briaApiJson['transactionMetadata']['statusMessage'] = 'IDP Extraction Succesful'
#    else :
#        briaApiJson['transactionMetadata']['statusMessage'] = message
#        briaApiJson['transactionMetadata']['status'] = 'Failed'

    if briaApiJson['generalInformation']['phoneNumber'].strip() == '':
        briaApiJson['generalInformation']['phoneNumber'] = '9999999999'

    if briaApiJson['generalInformation']['faxNumber'].strip() == '':
        briaApiJson['generalInformation']['faxNumber'] = '9999999999'

    if briaApiJson['generalInformation']['authorizationContactName'].strip() == '' or len(briaApiJson['generalInformation']['authorizationContactName'].strip()) < 7 :
        briaApiJson['generalInformation']['authorizationContactName'] = 'UR Department'

    if briaApiJson['procedureInformation']['procedureStartDate'].strip() == '':
        briaApiJson['procedureInformation']['procedureStartDate'] = '01/01/0001'

#reset service provider state if addresslineone is not set
    if briaApiJson['providerDetails']['servicingProvider']['address']['addressLineOne'] == '' and briaApiJson['providerDetails']['servicingProvider']['address']['state'] != '':
        briaApiJson['providerDetails']['servicingProvider']['address']['state'] = ""            


    return(briaApiJson)


## PART 1 of code till here
#################### MAIN FLASK ROUTE###################

@facesheetextraction.route(
    '/facesheetextraction', methods=['POST'], endpoint='fn_facesheetextraction')
@validate_jwt
def fn_facesheetextraction():

    logging.basicConfig(level=logging.INFO,  format='%(asctime)s - %(levelname)s - %(message)s')

    logging.info('IDP-PRE-PROCESS : facesheetextraction :  facesheetextraction : Received a request')

    if request.method != 'POST':
        return jsonify(
                    message="Invalid Action- Should be POST",
                    category="error",
                    status=500)
    request_data = request.get_data()
    if request_data:
        
        if request_data is None:
            return jsonify(
                    message="Byte array is empty",
                    category="error",
                    status=400), 400
        try:
            bytesio_array = BytesIO(request_data)
            pipelineContextJson = json.load(bytesio_array)
        except Exception as ex:
            logging.error('IDP-PRE-PROCESS : facesheetextraction :  : Error decoding byte array: ' + str(ex))
            return jsonify(
                    message="Unexpected Error in Error decoding byte array:: " + str(ex),
                    category="error",
                    status=500), 500
        
        # load files
        try:
            auth_user = os.getenv('OSCP_NP_USER')
            auth_password = os.getenv('OSCP_NP_PASS')    

           
            logging.info('IDP-PRE-PROCESS : facesheetextraction :  facesheetextraction : Username is ' +  auth_user )
            if auth_user is None:
                print("Unable to read username", sys.exc_info()[0])
                return jsonify(
                        message="Unable to read username",
                        category="error",
                        status=500)

            if auth_password is None:
                print("Unable to read password", sys.exc_info()[0])
                return jsonify(
                        message="Unable to read password",
                        category="error",
                        status=500)
                  
            
            file_path = os.getenv('FACESHEETEXTRACTION_2827_MODELSPATH')    
            if file_path is None:
                print("Unable to find directory", sys.exc_info()[0])
                return jsonify(
                        message="Unable to find FACESHEETEXTRACTION_2827_MODELSPATH directory",
                        category="error",
                        status=500)                  
            logging.info('IDP-PRE-PROCESS : facesheetextraction :  facesheetextraction : Directory is ' +  file_path )                        
                        
#            file_path = './api/2827/'
            hospitalfile = os.path.join(file_path,"hospitals.txt")
            
            #ipcmgrid = os.path.join(file_path, "IPCMGrid.xlsx")
            ipcmgrid = os.path.join(file_path, "IPCMGrid.zip")
            ipcmgridCPF = os.path.join(file_path, "IPCMGridCPF.pickled")
            icd10_file = os.path.join(file_path, "icd10_codes.xlsx")
            
            fieldkeysFile = os.path.join(file_path, "fieldkeys.json")
            keyValueOverrideFile = os.path.join(file_path, "keyValueOverrides.json")
            briaApiJsonTemplateFile = os.path.join(file_path,"bria_idp_api_request_template.json")

                       
            with smbclient.open_file(fieldkeysFile, mode="r",username=auth_user, password=auth_password,share_access="r") as fieldkeysFD:
                fieldKeys = json.load(fieldkeysFD)

            with smbclient.open_file(briaApiJsonTemplateFile, mode="r",username=auth_user, password=auth_password,share_access="r") as briaApiJsonTemplateFD:
                briaApiJson = json.load(briaApiJsonTemplateFD)

            with smbclient.open_file(keyValueOverrideFile, mode="rb",username=auth_user, password=auth_password,share_access="r") as keyValueOverrideFD:
                keyValueOverrides = json.load(keyValueOverrideFD)
            

        
        except Exception as ex:
            logging.error('IDP-PRE-PROCESS : facesheetextraction :  facesheetextraction : Unexpected Error in facesheetextraction-1 request : ' + str(ex))
            return jsonify(
                        message="Unexpected Error in facesheetextraction: " + str(ex),
                        category="error",
                        status=420)
        
        
        try: 
            idpRequestID = pipelineContextJson['Request']['Id']  
            pipelineRawContent = getPipeLineRawContent(pipelineContextJson)
            pipelineOCRRawContent = getPipeLineOCRRawContent(pipelineContextJson)
            pipelineRawContentdoc=nlp(pipelineRawContent)                    

            pipelineCustomkeyValuePairs = getPipelineCustomkeyValuePairs(pipelineContextJson,fieldKeys)
            pipelinePreBuiltkeyValuePairs = getPipelinePreBuiltkeyValuePairs(pipelineContextJson,nlp,fieldKeys)
            pipelineCustomJson =  None
            if 'CustomJSON' in pipelineContextJson['PipelineOptions']:
                if pipelineContextJson['PipelineOptions']['CustomJSON'] is not None:
                    pipelineCustomJson = json.loads(pipelineContextJson['PipelineOptions']['CustomJSON'])

            message=""
            try:
                getFaxNumber(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in GetFaxNumber: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:     
                getContactPhoneNumber(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getContactPhoneNumber: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getCustomerId(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getCustomerId: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getAccountNum(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getAccountNum: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getDOB(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getDOB: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getPatientName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineCustomJson,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getPatientName: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getRequestType(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getRequestType: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getProcedureStartDate(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getProcedureStartDate: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getDiagnosisCodes(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,icd10_file,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getDiagnosisCodes: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getRequestingNPI(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getRequestingNPI: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getServicingfacilityName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,hospitalfile,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getServicingfacilityName: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getRequestingProvider(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,nlp,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getRequestingProvider: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:     
                getMRN(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getMRN: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
                    
            try:                         
                getTIN(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getTIN: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getAddress(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
                getGoogleMapsaddress(pipelineRawContent,briaApiJson)
            except Exception as ex:
                message = message + "/" + "Failure in getAddress: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:                         
                getRequestingProviderAddress(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getRequestingProviderAddress: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:     
                pass
#                       getAuthorizationContactName(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,fieldKeys,pipelineRawContentdoc,nlp,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getAuthorizationContactName: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:     
                getIPCMValues(ipcmgrid,briaApiJson,fieldKeys)
                getIPCMValuesCPF(ipcmgridCPF,briaApiJson,fieldKeys)
            except Exception as ex:
                message = message + "/" + "Failure in getIPCMValues: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass

            try:     
                facilitySpecifickeyValueOverrides(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,keyValueOverrides)
            except Exception as ex:
                message = message + "/" + "Failure in facilitySpecifickeyValueOverrides: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass


            try:     
                getStaticValues(pipelineContextJson,pipelineRawContent,pipelinePreBuiltkeyValuePairs,briaApiJson,idpRequestID,pipelineRawContentdoc,fieldKeys,pipelineOCRRawContent,pipelineCustomkeyValuePairs)
            except Exception as ex:
                message = message + "/" + "Failure in getStaticValues: " + str(ex)
                logging.info('FACE SHEET 2827 : extractFace sheet :  ID: %s ,  %s',idpRequestID,message)
                pass
            
            return jsonify(message=message, category="Success", briaApiModel = briaApiJson, status=200)
            
        except Exception as ex:
            logging.error('IDP-PRE-PROCESS : facesheetextraction :  facesheetextraction : Unexpected Error in facesheetextraction3: ' + str(ex))
            return jsonify(
                        message="Unexpected Error in facesheetextraction: " + str(ex),
                        category="error",
                        status=420)


# api_app

In [ ]:
#!/usr/bin/env python
from email.errors import FirstHeaderLineIsContinuationDefect
from flask import Flask, request, send_file, Blueprint
import os
import sys
import cv2
import numpy as np
import io
from PIL import Image
import base64
from flask.json import jsonify
import jwt
from ibm_db2 import IBM_DB2_Connection
from pdf2image import convert_from_bytes
from random import randint
import datetime
import multiprocessing
import pytesseract
import logging
if os.name != 'nt':
    from gunicornapplication import StandaloneApplication
# Comment these lines in if attempting to run locally on windows
# Requires .env file to exist in root directory
#else:
#    from dotenv import load_dotenv
#    from pathlib import Path
#    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
#    load_dotenv(dotenv_path=os.path.join(BASE_DIR, "..", ".env"))
from convert_png_to_pdf import convert_png_to_pdf
from convert_pdf_to_png import convert_pdf_to_png, convert_pdf_to_png_bytes
from annotate_pdf import annotate_pdf
from validate_jwt import validate_jwt
from digitize_cm_survey import digitize_cm_survey
from digitize_hphb_survey import digitize_hphb_survey
from digitize_oncology_support_survey import digitize_oncology_support_survey
from digitize_customer_survey import digitize_customer_survey
from split_pdf import split_pdf
from split_tiff import split_tiff
from covid_otc_claim_extract import covid_otc_claim_extract
from convert_tif_to_pdf import convert_tif_to_pdf
from footnotes_compare import footnotes_compare
from apply_translation_to_pdf import apply_translation_to_pdf
from extractPdqEmail import extractPdqEmail
from extractPdqChecklist import extractPdqChecklist
from convert_pdf_to_html import convert_pdf_to_html_bytes
from get_intl_claims_key_name import get_intl_claims_key_name
from expedited_appeals_info_extract import execute
from gov_appeals_classify_page import is_expedited_page
from gov_appeals_classify_page import merge_id_function
from gov_appeals_classify_page import claim_extraction
from facesheetextraction import facesheetextraction
from md_sbc_preprocess_img import sbc_preprocess_img
from contract_pdf_preprocess import contract_pdf_preprocess
import json

def number_of_workers():
    optimum_workes_cnt  = (multiprocessing.cpu_count() * 2) + 1
    return 9 if optimum_workes_cnt > 9 else optimum_workes_cnt


def preprocess(img):
    
    img = img.astype(np.float32)

    img /= 255.

    img_gaussian = cv2.GaussianBlur(img, (3,3),0)

    kernel = np.array([[1, 1, 1],[1, 0, 1],[1, 1, 1]])

    img_filtered = cv2.filter2D(img_gaussian, -1, kernel)

    height = img_filtered.shape[0]
    width = img_filtered.shape[1]

    img_comp_orig_filtered = np.ndarray((height, width))

    j = 0
    while j<height:
        i = 0
        while i<width:
            if img_filtered[j,i] >= (6.5):
            # if img_filtered[j,i] >= (7.25):
                img_comp_orig_filtered[j,i] = (1)
            else:
                img_comp_orig_filtered[j,i] = img[j,i]
            i = i+1
        j = j+1
    
    img_comp_orig_filtered = img_comp_orig_filtered.astype(
        np.uint8) * 255
    
    img_comp_orig_filtered_inv = cv2.bitwise_not(
        img_comp_orig_filtered)

    kernel_dilate = np.ones((1,2), np.uint8)

    img_comp_orig_filtered_inv_dilated = cv2.dilate(
        img_comp_orig_filtered_inv, kernel_dilate,iterations=1)

    img_comp_orig_filtered_dilated = cv2.bitwise_not(
        img_comp_orig_filtered_inv_dilated)

    img_comp_orig_filtered_dilated_resized = cv2.resize(
        img_comp_orig_filtered_dilated,
        (1024,1024),
        interpolation=cv2.INTER_AREA)
    
    return img_comp_orig_filtered_inv_dilated, \
        img_comp_orig_filtered_dilated_resized


app = Flask(__name__)
app.register_blueprint(convert_pdf_to_png)
app.register_blueprint(convert_png_to_pdf)
app.register_blueprint(convert_pdf_to_png_bytes)
app.register_blueprint(annotate_pdf)
app.register_blueprint(split_pdf)
app.register_blueprint(split_tiff)
app.register_blueprint(apply_translation_to_pdf)
app.register_blueprint(extractPdqEmail)
app.register_blueprint(extractPdqChecklist)
app.register_blueprint(convert_pdf_to_html_bytes)
app.register_blueprint(get_intl_claims_key_name)
app.register_blueprint(facesheetextraction)
app.register_blueprint(is_expedited_page)
app.register_blueprint(merge_id_function)
app.register_blueprint(claim_extraction)
app.register_blueprint(contract_pdf_preprocess)


@app.route('/')
@app.route('/health')
def health():
    logging.basicConfig(level=logging.INFO,  format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info('IDP-PRE-PROCESS : health :  Healthy')
    return {'message': 'Healthy'}  # This will return as JSON by default with a 200 status code

@app.route('/ocr', methods=['POST'], endpoint='ocr')
def ocr():
    if request.method == 'POST':
        img_sent = base64.b64decode(request.data)
        nparr = np.frombuffer(img_sent, np.uint8)
        img = cv2.imdecode(nparr, 0)
        config=('--l ebg --oem 3 --psm 8')
        pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
        return {'analyzeResult': pytesseract.image_to_string(img)}

@app.route('/preprocess', methods=['POST'], endpoint='upload_file')
@validate_jwt
def upload_file():

    if request.method == 'POST':
        img_sent = base64.b64decode(request.data)
        nparr = np.frombuffer(img_sent, np.uint8)
        img = cv2.imdecode(nparr, 0)

        _, preprocessed_resized_img = preprocess(img)
        img_pil = Image.fromarray(preprocessed_resized_img.astype('uint8'))
        file_object = io.BytesIO()

        img_pil.save(file_object,'png')
        file_object.seek(0)
        return send_file(
            file_object,
            as_attachment=True,
            download_name='response_img.png',
            mimetype='image/png')

@app.route(
    '/pdf2tiff', methods=['POST'], endpoint='convert_pdf_to_images')
@validate_jwt
def convert_pdf_to_images():

    if request.method == 'POST':
        try: 
            data = request.get_data()
        except:
            print("Unexpected error :", sys.exc_info()[0])
            raise

        try: 
            images = convert_from_bytes(
                data,
                dpi=200)
        except:
            print("Unexpected error :", sys.exc_info()[0])
            raise

        msecpart=datetime.datetime.now().strftime("%f")
        dir_path = os.path.dirname(os.path.realpath(__file__))
        output_folder = dir_path+f"/data-{randint(0,1000000000)}-{msecpart}"

        try:
            os.makedirs(output_folder)
            print(f"directory {output_folder} created successfully")
        except:
            print("Unexpected error :", sys.exc_info()[0])
            raise

        for index, image in enumerate(images):
            try:
                image.save(f'{output_folder}/image-converted-from-bytes{index}.tiff')
            except:
                print("Unexpected error :", sys.exc_info()[0])
                raise

            

        #Read all tiff images and base64 encode
        tif_images = os.listdir(output_folder)
        print(f"tif images saved {tif_images}")

        encoded_images = {}
        for index, tif_image in enumerate(tif_images):
            tif_image_full_path = f'{output_folder}/{tif_image}'
            
            print(f"tif image full path {tif_image_full_path}")
            
            with open(tif_image_full_path, "rb") as f:
                tif_file = base64.b64encode(f.read())
                tif_file_str = tif_file.decode("utf-8")

            try:
                encoded_images[index] = tif_file_str
            except:
                print(
                    "Unexpected error with creating obj to be returned:",
                    sys.exc_info()[0])
                raise

        # Deletes all saved images and returns a binary of TIFF images
        for tif_image in tif_images:
            tif_image_full_path = f'{output_folder}/{tif_image}'
            os.remove(tif_image_full_path) 
        
        try:
            os.rmdir(output_folder)
            print(f"directory {output_folder} removed successfully")
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print(f"failed to remove the directory {output_folder}")

        return jsonify(encoded_images)

@app.route('/digitizecmsurvey', methods=['POST'], endpoint='digitize_cm_survey_ep')
@validate_jwt
def digitize_cm_survey_ep():

    if request.method == 'POST':
        pdf_tiff_base64_str = request.get_json()

        if 'pdf_bytes' in pdf_tiff_base64_str:
            pdf_data_str = pdf_tiff_base64_str['pdf_bytes']

            pdf_data_str_encoded = pdf_data_str.encode('utf-8')
            pdf_data_bytes = base64.b64decode(pdf_data_str_encoded)
        
        
        if 'tiff_bytes' in pdf_tiff_base64_str:
            tiff_data_str = pdf_tiff_base64_str['tiff_bytes']

            tiff_data_str_encoded = tiff_data_str.encode('utf-8')
            tiff_data_bytes = base64.b64decode(tiff_data_str_encoded)

        return digitize_cm_survey(pdf_data_bytes, tiff_data_bytes)   

@app.route('/digitizehphbsurvey', methods=['POST'], endpoint='digitize_hphb_survey_ep')
@validate_jwt
def digitize_hphb_survey_ep():

    if request.method == 'POST':
        pdf_tiff_base64_str = request.get_json()

        if 'pdf_bytes' in pdf_tiff_base64_str:
            pdf_data_str = pdf_tiff_base64_str['pdf_bytes']

            pdf_data_str_encoded = pdf_data_str.encode('utf-8')
            pdf_data_bytes = base64.b64decode(pdf_data_str_encoded)
        
        
        if 'tiff_bytes' in pdf_tiff_base64_str:
            tiff_data_str = pdf_tiff_base64_str['tiff_bytes']

            tiff_data_str_encoded = tiff_data_str.encode('utf-8')
            tiff_data_bytes = base64.b64decode(tiff_data_str_encoded)

        return digitize_hphb_survey(pdf_data_bytes, tiff_data_bytes)  

@app.route('/digitizeoncologysurvey', methods=['POST'], endpoint='digitize_oncology_support_survey_ep')
@validate_jwt
def digitize_oncology_support_survey_ep():

    if request.method == 'POST':
        pdf_tiff_base64_str = request.get_json()

        if 'pdf_bytes' in pdf_tiff_base64_str:
            pdf_data_str = pdf_tiff_base64_str['pdf_bytes']

            pdf_data_str_encoded = pdf_data_str.encode('utf-8')
            pdf_data_bytes = base64.b64decode(pdf_data_str_encoded)
        
        
        if 'tiff_bytes' in pdf_tiff_base64_str:
            tiff_data_str = pdf_tiff_base64_str['tiff_bytes']

            tiff_data_str_encoded = tiff_data_str.encode('utf-8')
            tiff_data_bytes = base64.b64decode(tiff_data_str_encoded)

        return digitize_oncology_support_survey(pdf_data_bytes, tiff_data_bytes)  

@app.route('/digitizesurveyforms', methods=['POST'], endpoint='digitize_customer_survey_ep')
@validate_jwt
def digitize_customer_survey_ep():

    if request.method == 'POST':
        pdf_tiff_base64_str = request.get_json()

        if 'pdf_bytes' in pdf_tiff_base64_str:
            pdf_data_str = pdf_tiff_base64_str['pdf_bytes']

            pdf_data_str_encoded = pdf_data_str.encode('utf-8')
            pdf_data_bytes = base64.b64decode(pdf_data_str_encoded)
        
        
        if 'tiff_bytes' in pdf_tiff_base64_str:
            tiff_data_str = pdf_tiff_base64_str['tiff_bytes']

            tiff_data_str_encoded = tiff_data_str.encode('utf-8')
            tiff_data_bytes = base64.b64decode(tiff_data_str_encoded)

        return digitize_customer_survey(pdf_data_bytes, tiff_data_bytes)   

@app.route('/covidotcclaimextract', methods=['POST'], endpoint='covid_otc_claim_extract_ep')
@validate_jwt
def covid_otc_claim_extract_ep():
    
    if request.method == 'POST':
        # img_bytes_str_encoded = request.data.encode('utf-8')
        img_bytes = base64.b64decode(request.data)
    
    return covid_otc_claim_extract(img_bytes)  


@app.route('/contractfncompare', methods=['POST'], endpoint='contract_footnotes_compare_ep')
@validate_jwt
def contract_footnotes_compare_ep():

    if request.method == 'POST':
        key_value_pair_loaded = request.get_json()

        old_footnotes = {}
        new_footnotes = {}
        
        for k,v in key_value_pair_loaded.items():
            if k == 'OldFootNotes':
                for old_footnotes_key, old_footnotes_value in v.items():
                    old_footnotes_key_int = int(old_footnotes_key)
                    old_footnotes[old_footnotes_key_int] = old_footnotes_value
            if k == 'NewFootNotes':
                for new_footnotes_key, new_footnotes_value in v.items():
                    new_footnotes_key_int = int(new_footnotes_key)
                    new_footnotes[new_footnotes_key_int] = new_footnotes_value

    try:    
        comp_results = footnotes_compare(
            old_footnotes,
            new_footnotes)

    except Exception as err:
        logging.error(
            f'FOOTNOTES-COMPARISON : contract_footnotes_compare : '
            f'Unexpected error in footnotes_compare : {str(err)}')

        comp_results = {
            'comparison_status':'failed',
            'error_type': (
                f'Unexpected error with contract_footnotes_compare'),
            'error_message': str(err)}

    finally:
        return comp_results


@app.route('/db2_methods', methods=['POST'], endpoint='db2_methods')
@validate_jwt
def db2_methods():
    '''
    API endpoint for all DB2 methods.  The POST call expects the body to be JSON formatted with the following objects:
        
        - 'connection'  - The DB2 connection string (TODO:  Look at adding this as an environment variable.).
        - 'data1'       - The first data object expected to to be supplied to the intended method.
        - 'method'      - The intended method to be called.

    Returns:
        The response from the intended method.
    '''

    required_fields = [ 'connection', 'data1', 'method' ]

    if request.method == 'POST':
        elements = request.get_json()

        if not all(element in elements for element in required_fields):
            return f"Missing one or more required fields: {required_fields}", 400
        if not 'life' in elements['connection'].lower():
            return f"Invalid connection string", 400

        ibm = IBM_DB2_Connection(elements['connection'], "", "")

        try:
            method = elements['method']
            logging.info(f'Attempting to perform (' + method + ')')
            if method == 'GetMemberInfo':
                res = ibm.check_for_member(elements['data1'])
                logging.info(f'Returning HTTP 200')
                return { "result": res }, 200

            elif method == 'WriteCoversheetInfo':
                res = ibm.insert_coversheet(elements['data1'])
                logging.info(f'Returning HTTP 200')
                return { "result": res }, 200

            elif method == 'InsertItemizationData':
                res = ibm.insert_itemizations(elements['data1'])
                logging.info(f'Returning HTTP 200')
                return { "result": res }, 200

            else:
                return f"method value provided: (" + elements['method'] + ') has not been implemented.', 400

        except ValueError as e:
            logging.error(f'{e}')
            return str(e), 400


@app.route(
    '/expdappealext', 
    methods=['POST'], 
    endpoint='expd_appeal_extract_ep')
    
@validate_jwt
def expd_appeal_extract_ep():

    if request.method == 'POST':
        image_wise_extract = request.get_json()

        try:    
            if len(image_wise_extract) == 1:
                for img_name, page_level_extracts_list in image_wise_extract.items():
                    _, _, _,extract_results = \
                        execute(
                            img_name, page_level_extracts_list )

        except Exception as err:
            logging.error(
                f'EXPEDITED-APPEALS-INFO-EXTRACT : expedited_appeals_info_extract : '
                f'Unexpected error in execute: {str(err)}')

            extract_results = {
                'comparison_status':'failed',
                'error_type': (
                    f'Unexpected error with expedited_appeals_info_extract'),
                'error_message': str(err)}

        finally:
            return json.dumps(extract_results)


@app.route('/sbcpreprocess', methods=['POST'], endpoint='sbc_preprocess_ep')
@validate_jwt
def sbc_preprocess_ep():
    
    if request.method == 'POST':
        base64_str = request.data

        
        try:    
          results = sbc_preprocess_img(base64_str)  

        except Exception as err:
            logging.error(
                f'SBC-PREPROCESS-IMG : sbc_preprocess_ep : '
                f'Unexpected error in execute: {str(err)}')

            results = {
                'img_preprocess_status':'failed',
                'error_type': (
                    f'Unexpected error with sbc_preprocess_img'),
                'error_message': str(err)}

        finally:
            return results


if __name__ == '__main__':

    if os.name == 'nt':
        app.run()
    else:
        if os.environ.get('TLS_CERT') is not None and os.environ.get('TLS_KEY') is not None :
            tls_cert_path = os.getenv('TLS_CERT')
            tls_key_path = os.getenv('TLS_KEY')
           
            options = {
            'bind': '%s:%s' % ('0.0.0.0', '5000'),
            'workers': number_of_workers(),
            #'workers':2,
            # 'threads':4,
            'max_requests' : os.getenv('APP_MAX_REQUESTS', 1),
            'reload': os.getenv('DEBUG_MODE', False),
    #        'worker_tmp_dir' : '/dev/shm',
            'certfile': tls_cert_path,
            'keyfile': tls_key_path,
            'timeout' : 600,
            'loglevel': 'info'
            }
            StandaloneApplication(app, options).run()
            
        else:
            options = {
            'bind': '%s:%s' % ('0.0.0.0', '5000'),
            # 'workers':2,
            # 'threads':4,
            'workers': number_of_workers(),
            'max_requests' : os.getenv('APP_MAX_REQUESTS', 1),
    #        'worker_tmp_dir' : '/dev/shm',
            'reload': os.getenv('DEBUG_MODE', False),
            'timeout' : 600,
            'loglevel': 'info'
            }
            StandaloneApplication(app, options).run()  
